<a href="https://colab.research.google.com/github/siroros/BI_SCRIPT/blob/master/Billing_tool_acct_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from google.colab import drive
import re 
import string
drive.mount('/content/drive/')


auth.authenticate_user()
print('Authenticated')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Authenticated


In [57]:
%%bigquery --project dhh---analytics-apac df_billing


with cc AS ( 
  with x as(
    SELECT
      DISTINCT c.client_id,
      (standard_commission_value * 100) as  stv,
      CAST(start_date_utc AS date) commission_start_date,
      invoicing_period --,
     -- tier_base,
     -- restaurant_revenue,
     --   DATE_SUB(CAST(LEAD(start_date_utc) OVER (PARTITION BY v.vendor_code ORDER BY start_date_utc ASC) AS date), INTERVAL 1 day) AS end_date
     FROM ml_billing_latest.commission c
     LEFT JOIN
      ml_billing_latest.commission_tier ct
    ON
      ct.commission_id = c.id
      AND ct.rdbms_id = 17
      
    WHERE TRUE
    and CAST(start_date_utc AS date) <= current_date() and (CAST(end_date_utc AS date) is null or CAST(end_date_utc AS date) >= current_date())
      -- end_Date_utc is null and
      --is_active
     -- and (lower_bound = 0 or lower_bound is null)
      AND c.rdbms_id = 17)
      
      select
      client_id,
    commission_start_date,
    invoicing_period,
      string_agg(cast(stv as string),',') as stv,
      count(distinct stv) as total_tiers_b
      from x
      
      group by 1,2,3),
      
contact as (
select
client_id,
co.name,
co.email,
updated_at,
row_number() over (partition by client_id order by updated_at desc) as rank

from ml_billing_latest.contact co

where rdbms_id = 17

)



select
distinct
c.name as client_name, 
billing_address, 
tax_id, 
bank_account_number,
bank_name, 
commission_start_date,
stv,
total_tiers_b,
invoicing_period,
co.name as contact_name,
co.email,
v.name as vendor_name,
v.vendor_code,
'Tax Exempt' as vat,
'No' as wht,
salesforce_id,
sf_account_id,
chain_code

from ml_billing_latest.vendor v

left join ml_billing_latest.client c
on c.id = v.client_id and v.rdbms_id = c.rdbms_id

left join cc on cc.client_id = c.id

left join contact co on co.client_id = c.id and co.rank = 1

left join (select vendor_code, chain_code, salesforce_id,sf_account_id from pandata.dim_vendors where rdbms_id = 17)sf on 
v.vendor_code = sf.vendor_code 

where v.rdbms_id = 17 and c.active =1
order by total_tiers_b desc



In [58]:
################## SELECT SOME COLUMNS IN BILLING TOOLS #########################
df_billing.info()
df_billing_new = df_billing[["client_name", "tax_id", "bank_account_number","bank_name","vendor_code","vendor_name","salesforce_id","sf_account_id","stv","total_tiers_b"]]
df_billing_new["bank_account_number_swap"] = df_billing_new["bank_name"]
df_billing_new["bank_name_swap"] = df_billing_new["bank_account_number"]
df_unique = df_billing_new.nunique()
df_billing_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150382 entries, 0 to 150381
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   client_name            150381 non-null  object
 1   billing_address        150238 non-null  object
 2   tax_id                 150382 non-null  object
 3   bank_account_number    150377 non-null  object
 4   bank_name              150348 non-null  object
 5   commission_start_date  150382 non-null  object
 6   stv                    150382 non-null  object
 7   total_tiers_b          150382 non-null  int64 
 8   invoicing_period       150382 non-null  object
 9   contact_name           150292 non-null  object
 10  email                  150375 non-null  object
 11  vendor_name            150382 non-null  object
 12  vendor_code            150382 non-null  object
 13  vat                    150382 non-null  object
 14  wht                    150382 non-null  object
 15  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150382 entries, 0 to 150381
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   client_name               150381 non-null  object
 1   tax_id                    150382 non-null  object
 2   bank_account_number       150377 non-null  object
 3   bank_name                 150348 non-null  object
 4   vendor_code               150382 non-null  object
 5   vendor_name               150382 non-null  object
 6   salesforce_id             150202 non-null  object
 7   sf_account_id             150215 non-null  object
 8   stv                       150382 non-null  object
 9   total_tiers_b             150382 non-null  int64 
 10  bank_account_number_swap  150348 non-null  object
 11  bank_name_swap            150377 non-null  object
dtypes: int64(1), object(11)
memory usage: 13.8+ MB


In [59]:
#### CONVERT STV TO ARRAY #####
def make_array(stv) :
  arr = stv.split(',')
  finalArray = np.asarray(arr, dtype = np.float64,  
                        order ='C') 

  #arr = np.float(arr)
  #output = ["%.2f" % elem for elem in arr]
  return(finalArray)
df_billing_new["stv"] = df_billing_new["stv"].apply(make_array)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [60]:
######################### VALIDATE BILLING TOOL ########################
def check_character(txt):
  r =  re.findall(r"[@_!#$%^&*()<>?/\|-}{~:a-zA-Z]", txt)
  if len(r) == 0 :
   return 0
  else:
   return 1

def check_space(col):
  c = re.findall("[' ']", col)
  if len(c) == 0 :
   return 0
  else:
   return 1

def strip_special_char(dt):
  r = re.compile(r"[@_!#$%^&*()<>?/\|-}{~:a-zA-Z]")
  dt = r.sub('', dt)
  r = re.compile(r"'")
  dt = r.sub('', dt)
  r = re.compile(' ')
  dt = r.sub('', dt)
  r = re.compile('-')
  dt = r.sub('',dt)
  return dt

def check_len  (col, len_start, len_end):
  if col >= len_start and col <= len_end:
    return 0
  else :
    return 1

def check_null (col):
  if col == None or col == True  :
    return 1
  else : 
    return 0
 
def cast_to_none(x):
  if (x is None) or (x == ''):
    return np.nan
  return x


df_billing_new["tax_id"] = df_billing_new["tax_id"].apply(cast_to_none)
df_billing_new["bank_account_number"] = df_billing_new["bank_account_number"].apply(cast_to_none)
df_billing_new["len_tax_id"] = df_billing_new['tax_id'].astype(str).str.len()
df_billing_new["is_tax_contain_char"] =   df_billing_new["tax_id"].apply(check_character)
df_billing_new["is_tax_contain_space"] =   df_billing_new["tax_id"].apply(check_space)
df_billing_new["is_tax_id_len_correct"] = df_billing_new["len_tax_id"].apply(check_len, args = (10,13))
df_billing_new["len_acc_no"] = df_billing_new["bank_account_number"].astype(str).str.len()
df_billing_new["is_acc_no_len_correct"] = df_billing_new["len_acc_no"].apply(check_len, args = (10,15))
df_billing_new["is_acc_contain_char"] =   df_billing_new["bank_account_number"].astype(str).apply(check_character)
df_billing_new["is_acc_contain_space"] =   df_billing_new["bank_account_number"].astype(str).apply(check_space)
df_billing_new["new_tax_id"] = df_billing_new["tax_id"].apply(strip_special_char)
df_billing_new["new_acc_no"] = df_billing_new["bank_account_number"].astype(str).apply(strip_special_char)
df_billing_new["new_len_tax_id"] = df_billing_new["new_tax_id"].astype(str).str.len()
df_billing_new["new_len_acc_no"] = df_billing_new["new_acc_no"].astype(str).str.len()
df_billing_new["is_new_tax_id_len_correct"] = df_billing_new["new_len_tax_id"].apply(check_len, args = (10,13))
df_billing_new["is_new_acc_no_len_correct"] = df_billing_new["new_len_acc_no"].apply(check_len, args = (10,15))
df_billing_new["is_tax_id_null"] = df_billing_new["tax_id"].isnull().apply(check_null) 
df_billing_new["is_acc_no_null"] = df_billing_new["bank_account_number"].isnull().apply(check_null)
df_billing_new["is_salesforce_id_null"] = df_billing_new["salesforce_id"].isnull().apply(check_null)
df_billing_new["is_sf_account_id_null"] = df_billing_new["sf_account_id"].isnull().apply(check_null)
df_billing_new["is_tax_id_pass_at_the_begginning"] = np.where(df_billing_new["tax_id"] == df_billing_new["new_tax_id"] , 0 ,1)
df_billing_new["is_acc_no_pass_at_the_begginning"] = np.where(df_billing_new["bank_account_number"] == df_billing_new["new_acc_no"] , 0 ,1)
df_billing_new["bank_account_number_swap_clean"] =  df_billing_new["bank_account_number_swap"].astype(str).apply(strip_special_char)
df_billing_new["bank_account_number_swap_clean"] = df_billing_new["bank_account_number_swap_clean"].apply(cast_to_none)
df_billing_new["len_acc_no_swap"] = df_billing_new["bank_account_number_swap_clean"].astype(str).str.len()
df_billing_new["is_len_acc_no_swap_correct"]  = df_billing_new["len_acc_no_swap"].apply(check_len, args = (10,15))
df_billing_new["is_teir_excceed_5"] = np.where(df_billing_new["total_tiers_b"] > 5 , 1,0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [61]:
df_billing_new["is_tax_id_pass_after_cleaning"]  = np.where(df_billing_new["is_new_tax_id_len_correct"] == 0 , 0 ,1)
df_billing_new["is_acc_no_pass_after_cleaning"] = np.where(df_billing_new["is_new_acc_no_len_correct"] == 0 , 0 ,1)

conditions_billing = [
    (df_billing_new["is_tax_id_pass_at_the_begginning"] == 0 ),
    (df_billing_new["is_new_tax_id_len_correct"] == 0),
    (df_billing_new["is_tax_id_pass_at_the_begginning"] == 1 ) & (df_billing_new["is_new_tax_id_len_correct"] == 1)
    ]
values = ['pass_at_the_beginning', 'pass_after_cleaning', 'not_pass']

conditions_acct_no = [
    (df_billing_new["is_acc_no_pass_at_the_begginning"] == 0 ),
    (df_billing_new["is_new_acc_no_len_correct"] == 0),
    (df_billing_new["is_acc_no_pass_at_the_begginning"] == 1 ) & (df_billing_new["is_new_acc_no_len_correct"] == 1)
    ]

df_billing_new["tax_id_status"]  = np.select(conditions_billing, values)
df_billing_new["acc_no_status"]  = np.select(conditions_acct_no, values)

**Plot the distributuion of billing tool**

**Sales Force Data Checkking**

Merge with Win's scripts

In [62]:

##bankname_path = 'https://docs.google.com/spreadsheets/d/1BfueEqYY9av91GDi8PWM76IqrOrYQ-wp_vKVWUH99Gk/edit#gid=60220015'
##commission_path = 'https://docs.google.com/spreadsheets/d/1ldnCfmms61-q1j1WE5zX4xcy5wTeIsc2p-udNYckSOw/edit#gid=444608842'
##tax_path = 'https://docs.google.com/spreadsheets/d/1xJBuiL2Mx7CZDG9evLvduS0quzwwN3jBTEfVqrl4iLU/edit#gid=1966818440'

bankname_path = 'https://docs.google.com/spreadsheets/d/1QdiabCaha2LQImwv76yLzPFXI_hjXrMy3HJbR9DoGRw/edit#gid=342813547'
commission_path = 'https://docs.google.com/spreadsheets/d/1BF3i10x27pbdkXVx4Wxe6OB5GS7DIzCpK8zeTyHbMfU/edit#gid=1091098651'
tax_path = 'https://docs.google.com/spreadsheets/d/1fQ1BYJdGXaDsQiIBGsgg6LJ1roTWsu0BctFTGqptWnE/edit#gid=679379116'
bank_master_path = 'https://docs.google.com/spreadsheets/d/1chrw0l1F_Owl4REtx67S3QEvgs9SmVk6LS6_tGaNh44/edit#gid=0'

gc = gspread.authorize(GoogleCredentials.get_application_default())
bankname = gc.open_by_url(bankname_path)
sheet1 = bankname.worksheet('Bank account no. and name')  ### BANK NAME is not consistant
data1 = sheet1.get_all_values()
df_bankname = pd.DataFrame(data1)
df_bankname.columns = df_bankname.iloc[0]
df_bankname = df_bankname.iloc[1:]
df_bankname = pd.DataFrame(df_bankname)
def cast_to_none(x):
  if (x is None) or (x == ''):
    return np.nan
  return x

df_bankname["GRID"]  = df_bankname["GRID"].apply(cast_to_none)
df_bankname["Bank Detail: ID"]  = df_bankname["Bank Detail: ID"].apply(cast_to_none)
df_bankname["Bank Account Number"]  = df_bankname["Bank Account Number"].apply(cast_to_none)
df_bankname["Bank Name"]  = df_bankname["Bank Name"].apply(cast_to_none)
df_bankname["Account Status"]  = df_bankname["Account Status"].apply(cast_to_none)
df_bankname["Status"]  = df_bankname["Status"].apply(cast_to_none)

In [63]:
bank_master = gc.open_by_url(bank_master_path)
sheet4 = bank_master.worksheet('Sheet1')  
data4 = sheet4.get_all_values()
df_bank_master = pd.DataFrame(data4)
df_bank_master.columns = df_bank_master.iloc[0]
df_bank_master = df_bank_master.iloc[1:]
df_bank_master = pd.DataFrame(df_bank_master)

In [64]:
##SLICE DATA FOR SF BANK NAME AND BILLING
df_bank_sf = df_bank_master.iloc[:,0:4]
df_bank_billing = df_bank_master.iloc[:,5:9]

In [65]:

##REMOVE NULL ROWS##
def cast_to_none(x):
  if (x is None) or (x == ''):
    return np.nan
  return x
df_bank_sf["SF bank name"]  = df_bank_sf["SF bank name"].apply(cast_to_none)
df_bank_sf["Clean name"] =df_bank_sf["Clean name"].apply(cast_to_none)
df_bank_sf["chk"] =df_bank_sf["chk"].apply(cast_to_none)
df_bank_sf["name for upload"] =df_bank_sf["name for upload"].apply(cast_to_none)
df_bank_billing["BT bank name"] = df_bank_billing["BT bank name"].apply(cast_to_none)
df_bank_billing["Clean name"] = df_bank_billing["Clean name"].apply(cast_to_none)
df_bank_billing["name for upload"] = df_bank_billing["name for upload"].apply(cast_to_none)
df_bank_billing["chk"] = df_bank_billing["chk"].apply(cast_to_none)
#### DROP EMPTHY ROW #######
df_bank_sf_new = df_bank_sf
df_bank_billing_new = df_bank_billing

values_not_empty = ['Y']

conditions_not_empty = [
                      (df_bank_billing_new["BT bank name"].isnull() & 
                       df_bank_billing_new["Clean name"].isnull() & 
                       df_bank_billing_new["name for upload"].isnull() &
                       df_bank_billing_new["chk"].isnull())]

df_bank_billing_new ["chk_empty_row"]  = np.select(conditions_not_empty, values_not_empty)
df_bank_billing_new = df_bank_billing_new[(df_bank_billing_new ["chk_empty_row"] != 'Y' )]


##### DROP EMPTY ROW FOR salesforce #####

conditions_not_empty_sf = [
                      (df_bank_sf_new["SF bank name"].isnull() & 
                       df_bank_sf_new["Clean name"].isnull() & 
                       df_bank_sf_new["name for upload"].isnull() &
                       df_bank_sf_new["chk"].isnull())]

df_bank_sf_new ["chk_empty_row"]  = np.select(conditions_not_empty_sf, values_not_empty)
df_bank_sf_new = df_bank_sf_new[(df_bank_sf_new ["chk_empty_row"] != 'Y' )]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [66]:
### DATA QUALITY CHECKING ###
#### TOTAL ROWS####
print(len(df_bank_sf_new.index))
#### UNIQUE VALUE ####
print(df_bank_sf_new.nunique())
#### SF BANK NAME DUPLICATE ####
print(df_bank_sf_new["SF bank name"].duplicated().sum())
df_bank_sf_new[df_bank_sf_new.duplicated()]

#### TOTAL ROWS BL #####
print(len(df_bank_billing_new.index))
#### UNIQUE VALUE ####
print(df_bank_billing_new.nunique())
#### BILLING BANK NAME DUPLICATE ####
print(df_bank_billing_new["BT bank name"].duplicated().sum())
df_bank_billing_new[df_bank_sf_new.duplicated()]

### DROP DUPLICATE ROWS ####
df_bank_sf_unique = df_bank_sf_new.drop_duplicates()
df_bank_billing_unique = df_bank_billing_new.drop_duplicates()
### CHECK WHETHER STILL HAVE DUPLICATE BANK NAME OR NOT ####
print(df_bank_billing_unique["BT bank name"].duplicated().sum())
print(df_bank_sf_unique["SF bank name"].duplicated().sum())



697
0
SF bank name       671
Clean name          20
chk                 21
name for upload     20
chk_empty_row        1
dtype: int64
25
310
0
BT bank name       306
Clean name          21
chk                 21
name for upload     20
chk_empty_row        1
dtype: int64
4
0
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [67]:
df_bank_sf_unique.head()
df_bank_sf_unique.info()
df_bank_sf_unique.head()
df_bank_billing_unique.info()
df_bank_billing_unique.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 1 to 697
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SF bank name     671 non-null    object
 1   Clean name       669 non-null    object
 2   chk              672 non-null    object
 3   name for upload  672 non-null    object
 4   chk_empty_row    672 non-null    object
dtypes: object(5)
memory usage: 31.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 1 to 310
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BT bank name     306 non-null    object
 1   Clean name       305 non-null    object
 2   chk              306 non-null    object
 3   name for upload  306 non-null    object
 4   chk_empty_row    306 non-null    object
dtypes: object(5)
memory usage: 14.3+ KB


,BT bank name,Clean name,chk,name for upload,chk_empty_row
1,004: KASIKORNBANK,KBANK,KBANK,KBANK (Kasikorn),0
2,014: SIAM COMMERCIAL BANK,SCB,SCB,SCB (Siam Commercial Bank),0
3,TMB (TMB Bank),TMB,TMB,TMB (TMB Bank),0
4,002: BANGKOK BANK,BBL,BBL,BBL (Bangkok Bank),0
5,KTB (KrungThai Bank),KTB,KTB,KTB (KrungThai Bank),0


In [68]:
### RENAME COLUMN NAME #####
df_bank_sf_unique = df_bank_sf_unique.rename(columns = {'Clean name': 'clean_name_sf','chk':'chk_sf','name for upload':'name_for_upload_sf'}, inplace = False)
df_bank_billing_unique = df_bank_billing_unique.rename(columns = {'Clean name': 'clean_name_B','chk':'chk_B','name for upload':'name_for_upload_B'}, inplace = False)
df_bank_sf_unique = df_bank_sf_unique.iloc[:,0:4]
df_bank_billing_unique = df_bank_billing_unique.iloc[:,0:4]
df_bank_sf_unique.info()
df_bank_billing_unique.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 672 entries, 1 to 697
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SF bank name        671 non-null    object
 1   clean_name_sf       669 non-null    object
 2   chk_sf              672 non-null    object
 3   name_for_upload_sf  672 non-null    object
dtypes: object(4)
memory usage: 26.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 306 entries, 1 to 310
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   BT bank name       306 non-null    object
 1   clean_name_B       305 non-null    object
 2   chk_B              306 non-null    object
 3   name_for_upload_B  306 non-null    object
dtypes: object(4)
memory usage: 12.0+ KB


In [69]:
### IMPORT COMMISSION FILE and GET FIRST ROW AS HEADER ####
def cast_to_none(x):
  if (x is None) or (x == ''):
    return np.nan
  return x
commission_path = 'https://docs.google.com/spreadsheets/d/1BF3i10x27pbdkXVx4Wxe6OB5GS7DIzCpK8zeTyHbMfU/edit#gid=1091098651'
commission = gc.open_by_url(commission_path)
sheet2 = commission.worksheet('Commission')
data2 = sheet2.get_all_values()
df_commission = pd.DataFrame(data2)
df_commission.columns = df_commission.iloc[0]
df_commission = df_commission.iloc[1:]
df_commission = pd.DataFrame(df_commission)

####IMPORT TAX FILE AND GET FIRST ROW AS HEADER ####
tax = gc.open_by_url(tax_path)
sheet3 = tax.worksheet('Tax ID') ### Tax sheet name was changed from previous file
data3 = sheet3.get_all_values()
df_tax = pd.DataFrame(data3)
df_tax.columns = df_tax.iloc[0]
df_tax = df_tax.iloc[1:]
df_tax = pd.DataFrame(df_tax)


In [70]:

df_tax.info()
df_commission["GRID"] = df_commission["GRID"].apply(cast_to_none)
df_commission["Contract ID"] = df_commission["Contract ID"].apply(cast_to_none)
df_commission["Contract Number"] = df_commission["Contract Number"].apply(cast_to_none)
df_commission["Tier ID"] = df_commission["Tier ID"].apply(cast_to_none)
df_commission["Tier Name"] = df_commission["Tier Name"].apply(cast_to_none)
df_commission.columns = ['GRID', 'Contract ID', 'Contract Number', 'Commission in %', 'Tier ID', 'Tier Name', 'Commission_tier_pc']
df_commission["Commission in %"] = df_commission["Commission in %"].apply(cast_to_none)
df_commission["Commission_tier_pc"] = df_commission["Commission_tier_pc"].apply(cast_to_none)
df_tax["Account ID"] = df_tax["Account ID"].apply(cast_to_none)
df_tax["Tax Number"] = df_tax["Tax Number"].apply(cast_to_none)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123335 entries, 1 to 123335
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   GRID        123335 non-null  object
 1   Account ID  123335 non-null  object
 2   Tax Number  123335 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


In [71]:
df_commission

,GRID,Contract ID,Contract Number,Commission in %,Tier ID,Tier Name,Commission_tier_pc
1,45ZPJH,8006900000AfSR9,2453553,NaN,a1P6900000GwcH7,a1P6900000GwcH7,20
2,45WLEQ,8001r000009V2tt,1739956,27.5,NaN,NaN,NaN
3,45AGIW,8001r00000ANsYt,1807834,25,NaN,NaN,NaN
4,45ANVR,8001r00000ANotq,1793757,30,NaN,NaN,NaN
5,45ZPJH,8006900000AfSR9,2453553,NaN,a1P6900000GwcH8,a1P6900000GwcH8,23
...,...,...,...,...,...,...,...
123763,HKQ34I,8006900000AfGdw,2399638,35,NaN,NaN,NaN
123764,HKXF9V,8006900000AvNcm,2393603,32,NaN,NaN,NaN
123765,HKXINE,8006900000AfQ8O,2438928,32,NaN,NaN,NaN
123766,HK1DTM,80069000009ZDEy,2465574,35,NaN,NaN,NaN


In [72]:
df_comm = df_commission
############# RENAME DUP COLUMN NAME ####################################


def MA(item):
  a = list(df_have_tier["Commission_tier_pc"][(df_have_tier["GRID"] == item)]) 
  a = np.asarray(a, dtype = np.float64,  
                        order ='C') 
  return a

########## MAKE ARRAY FOR COMMISSION TIER #######################
##STEP 1 SELECT ONLY VENDORS WHO HAVE TIER TO REDUCE CHUCK SIZE OF DATA ########
df_have_tier = df_comm[["GRID","Commission_tier_pc"]]
df_have_tier["Commission_tier_pc"] = pd.to_numeric(df_have_tier["Commission_tier_pc"])
df_have_tier = df_have_tier[(df_have_tier['Commission_tier_pc'].notnull())]
df_have_tier["list_commission_tier"] = df_have_tier["GRID"].apply(MA)
df_have_tier = df_have_tier[["GRID","Commission_tier_pc","list_commission_tier"]]
##STEP 2 MAP BACK TO df_commission ########
df_comm = pd.merge(df_comm,df_have_tier, how= "left", left_on = "GRID", right_on= "GRID", suffixes=(None,'_t'))
df_comm["total_tiers"] = df_comm["list_commission_tier"].str.len()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
### TIER 1 ####
df_comm[(df_comm["total_tiers"] == 1)]

,GRID,Contract ID,Contract Number,Commission in %,Tier ID,Tier Name,Commission_tier_pc,Commission_tier_pc_t,list_commission_tier,total_tiers
60720,45ALPU,8001r000009Uwee,1716503,NaN,a1P1r00000GwMYz,TH Tiered Commission - Incremental GMV 1,30,30.0,[30.0],1.0


In [74]:
df_have_tier

,GRID,Commission_tier_pc,list_commission_tier
1,45ZPJH,20.0,"[20.0, 23.0, 25.0, 27.0, 30.0]"
5,45ZPJH,23.0,"[20.0, 23.0, 25.0, 27.0, 30.0]"
9,45ZPJH,25.0,"[20.0, 23.0, 25.0, 27.0, 30.0]"
13,45ZPJH,27.0,"[20.0, 23.0, 25.0, 27.0, 30.0]"
17,45ZPJH,30.0,"[20.0, 23.0, 25.0, 27.0, 30.0]"
...,...,...,...
100149,45J3KP,29.0,"[24.0, 29.0]"
110149,45788V,27.0,"[27.0, 32.0]"
110150,45788V,32.0,"[27.0, 32.0]"
110151,457NFV,32.0,"[32.0, 27.0]"


In [75]:
####### DROP DUPLICATE DATA FROM BANK FILE #####
print(len(df_bankname))
df1=df_bankname[['GRID','Bank Account Number','Bank Name']]
print(len(df1.drop_duplicates()))
df_bank_unique =df1.drop_duplicates()
df_bank_unique

115538
114661


,GRID,Bank Account Number,Bank Name
1,45ZWID,2102113094,SCB (Siam Commercial Bank)
2,45WK2X,4240702408,SCB (Siam Commercial Bank)
3,45AH73,181146850,KBANK (Kasikorn)
4,45RL64,4402195358,SCB
5,45ZRFK,6341000198,KASIKORN (KBANK)
...,...,...,...
115534,HKIRVN,4240515386,KTB (KrungThai Bank)
115535,HKIR1Y,3702597149,SCB (Siam Commercial Bank)
115536,HKIPG1,221927362,KBANK (Kasikorn)
115537,HKIP8M,221927362,KBANK (Kasikorn)


In [76]:
###### REPLACE NULL 0 and combine 2 duplicate column name into 1 column #####
df_commission.info()
df_commission["Commission in %"] = (pd.to_numeric(df_commission["Commission in %"])).fillna(0)
df_commission["Commission_tier_pc"] = (pd.to_numeric(df_commission["Commission_tier_pc"])).fillna(0)
df_commission["Commission_tier"] = df_commission["Commission in %"] + df_commission["Commission_tier_pc"]
df_commission = df_commission[["GRID","Contract ID" ,"Contract Number" ,"Tier ID", "Tier Name", "Commission_tier",'Commission_tier_pc']]
df_commission= df_commission.rename(columns = {'Commission in %': 'Commission'}, inplace = False)

df_commission_sort1 = df_commission.sort_values(['GRID', 'Commission_tier'], ascending=[1, 1])
df_commission_sort2 = df_commission.sort_values(['GRID','Commission_tier_pc'], ascending=[1,1])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123767 entries, 1 to 123767
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   GRID                123767 non-null  object
 1   Contract ID         123767 non-null  object
 2   Contract Number     123767 non-null  object
 3   Commission in %     123548 non-null  object
 4   Tier ID             219 non-null     object
 5   Tier Name           219 non-null     object
 6   Commission_tier_pc  219 non-null     object
dtypes: object(7)
memory usage: 6.6+ MB


In [77]:
#######  COMBINE tax file and Bank file together by using outer join #######
def check_null (col):
  if col  == True or col is None  :
    return 1
  else : 
    return 0
  
df_tax
df_bank_unique
df_tax
df_merge_bank_tax= pd.merge(df_tax,df_bank_unique,left_on= 'GRID', right_on = 'GRID', how='outer', suffixes=('_tax','_bank' ))
df_merge_bank_tax["is_tax_null"] = df_merge_bank_tax["Tax Number"].isnull().apply(check_null)
df_merge_bank_tax["is_acc_no_null"] = df_merge_bank_tax["Bank Account Number"].isnull().apply(check_null)


In [78]:
print("Total records after match bank and tax", str(len(df_merge_bank_tax.index)))
print("Total tax null : ",str(df_merge_bank_tax["is_tax_null"].sum()))
print("Total acc null : ",str(df_merge_bank_tax["is_acc_no_null"].sum()))


Total records after match bank and tax 123335
Total tax null :  1184
Total acc null :  10320


In [79]:
######## RESTUCTURE COMMISSION TIER TO ARRAY ######
df_commission_tier_group = df_commission_sort1.groupby('GRID')['Commission_tier'].apply(np.array).reset_index(name='Commission_tier')

def len_comm (arr) :
  length = len(arr)
  return length

df_commission_tier_group["total_tiers"] = df_commission_tier_group["Commission_tier"].apply(len_comm)
# df_bankname_group= len(df_bankname_group)>1
# df_bankname_group[len(df_bankname_group['Bank Detail: ID']) > 1]
# df_tax_group['len_bank_detail'] = df_bankname_group['Bank Account Number'].apply(lambda x : len(x))
df_commission_tier_group.head(100)
df_merge_bank_tax_com = pd.merge(df_merge_bank_tax, df_commission_tier_group, how= 'outer', left_on='GRID', right_on= 'GRID', suffixes=(None,'_com'))
df_merge_bank_tax_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123475 entries, 0 to 123474
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   GRID                 123475 non-null  object 
 1   Account ID           123335 non-null  object 
 2   Tax Number           122151 non-null  object 
 3   Bank Account Number  113015 non-null  object 
 4   Bank Name            113032 non-null  object 
 5   is_tax_null          123335 non-null  float64
 6   is_acc_no_null       123335 non-null  float64
 7   Commission_tier      123184 non-null  object 
 8   total_tiers          123184 non-null  float64
dtypes: float64(3), object(6)
memory usage: 9.4+ MB


In [80]:
def len_comm (arr) :
  length = len(arr)
  return length

df_commission_tier_group["Commission_tier"].apply(len_comm)

0         1
1         1
2         3
3         1
4         1
         ..
123179    1
123180    1
123181    1
123182    1
123183    1
Name: Commission_tier, Length: 123184, dtype: int64

In [81]:
df_summary_selected_column = df_merge_bank_tax_com[['GRID','Bank Name','Bank Account Number', 'Commission_tier','total_tiers','Tax Number']]
df_summary_selected_column['Tax Number'] =  df_summary_selected_column['Tax Number'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
df_summary_selected_column

,GRID,Bank Name,Bank Account Number,Commission_tier,total_tiers,Tax Number
0,45ZWU8,NaN,NaN,[25.0],1.0,nan
1,45AAFR,NaN,NaN,[35.0],1.0,1100501097390
2,45ZWJD,NaN,NaN,[20.0],1.0,nan
3,45AAFP,NaN,NaN,[20.0],1.0,3100101029938
4,45ZWJH,NaN,NaN,[20.0],1.0,nan
...,...,...,...,...,...,...
123470,45WU34,NaN,NaN,[30.0],1.0,nan
123471,45ZCRS,NaN,NaN,[39.0],1.0,nan
123472,HK0XI7,NaN,NaN,[20.0],1.0,nan
123473,HKCB17,NaN,NaN,[32.0],1.0,nan


In [83]:
######################### VALIDATE ########################
######### FUNCTION CHECK SPECIAL CHAR #############
def check_character(txt):
  r =  re.findall(r"[@_!#$%^&*()<>?/\|-}{~:a-zA-Z]", txt)
  if len(r) == 0 :
   return 0
  else:
   return 1
######### FUNCTION CHECK SPACE #############
def check_space(col):
  c = re.findall("[' ']", col)
  if len(c) == 0 :
   return 0
  else:
   return 1

######## FUNCTION CLEAN SPECIAL CHAR AND REMOVE SPACE #######
def strip_special_char(dt):
  r = re.compile(r"[@_!#$%^&*()<>?/\|-}{~:a-zA-Z]")
  dt = r.sub('', dt)
  r = re.compile(r"'")
  dt = r.sub('', dt)
  r = re.compile(' ')
  dt = r.sub('', dt)
  r = re.compile('-')
  dt = r.sub('',dt)
  return dt

######## FUNCTION CHECK TOTAL DIGIT ##################
def check_len  (col, len_start, len_end):
  if col >= len_start and col <= len_end:
    return 0
  else :
    return 1
######## FUNCTION CHECK NULL OF A COLUMN #############
def check_null (col):
  if col == True or col is None :
    return 1
  else : 
    return 0

####### FUNCTION CHECK_NUMBER_COMMISSION_TIERS #######
def check_tier(tier):
  if tier == True:
    return 1
  else :
    return 0

df_summary_selected_column["len_tax_id"] = df_summary_selected_column["Tax Number"].str.len()
df_summary_selected_column["is_tax_contain_char"] = df_summary_selected_column["Tax Number"].astype(str).apply(check_character)
df_summary_selected_column["is_tax_contain_space"]=df_summary_selected_column["Tax Number"].astype(str).apply(check_space)
df_summary_selected_column["is_tax_id_len_correct"] = df_summary_selected_column['len_tax_id'].apply(check_len,args = (10,13))
df_summary_selected_column["len_acc_no"]=df_summary_selected_column["Bank Account Number"].astype(str).str.len()
df_summary_selected_column["is_acc_no_len_correct"] = df_summary_selected_column["len_acc_no"].apply(check_len , args =(10,15))

# df_summary_selected_column["is_acc_no_len_correct"]= df_summary_selected_column["len_acc_no"].apply(check_len, args =(10,15))
df_summary_selected_column["is_acc_contain_char"]=df_summary_selected_column["Bank Account Number"].astype(str).apply(check_character)
df_summary_selected_column["is_acc_contain_space"]=df_summary_selected_column["Bank Account Number"].astype(str).apply(check_space)
df_summary_selected_column["new_tax_id"]=df_summary_selected_column["Tax Number"].apply(strip_special_char)
df_summary_selected_column["new_acc_no"]=df_summary_selected_column["Bank Account Number"].astype(str).apply(strip_special_char)
df_summary_selected_column["new_len_tax_id"]=df_summary_selected_column["new_tax_id"].astype(str).str.len()
df_summary_selected_column["new_len_acc_no"]=df_summary_selected_column["new_acc_no"].astype(str).str.len()
df_summary_selected_column['is_new_tax_id_len_correct'] = df_summary_selected_column['new_len_tax_id'].apply(check_len ,args=(10,13))
df_summary_selected_column['is_new_acc_no_len_correct'] = df_summary_selected_column['new_len_acc_no'].apply(check_len ,args=(10,15))
df_summary_selected_column['is_tax_id_null'] = df_summary_selected_column['Tax Number'].isnull().apply(check_null)
df_summary_selected_column["is_acc_no_null"]=df_summary_selected_column["Bank Account Number"].isnull().apply(check_null)
df_summary_selected_column["is_tax_id_pass_at_the_beginning"] = np.where(df_summary_selected_column["Tax Number"] == df_summary_selected_column["new_tax_id"] , 0 ,1)
df_summary_selected_column["is_acc_no_pass_at_the_beginning"] = np.where(df_summary_selected_column["Bank Account Number"] == df_summary_selected_column["new_acc_no"] , 0 ,1)
#df_summary_selected_column['is_new_tax_id_len_correct'] = ( df_summary_selected_column['new_tax_id'].str.len() - df_summary_selected_column['len_tax_id'] ==0)
df_summary_selected_column["is_no_tier_correct"] = np.where(df_summary_selected_column["total_tiers"] > 5  , 1 ,0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [84]:
df_summary_selected_column.info()
df_summary_selected_column.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123475 entries, 0 to 123474
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   GRID                             123475 non-null  object 
 1   Bank Name                        113032 non-null  object 
 2   Bank Account Number              113015 non-null  object 
 3   Commission_tier                  123184 non-null  object 
 4   total_tiers                      123184 non-null  float64
 5   Tax Number                       123475 non-null  object 
 6   len_tax_id                       123475 non-null  int64  
 7   is_tax_contain_char              123475 non-null  int64  
 8   is_tax_contain_space             123475 non-null  int64  
 9   is_tax_id_len_correct            123475 non-null  int64  
 10  len_acc_no                       123475 non-null  int64  
 11  is_acc_no_len_correct            123475 non-null  int64  
 12  is

,GRID,Bank Name,Bank Account Number,Commission_tier,total_tiers,Tax Number,len_tax_id,is_tax_contain_char,is_tax_contain_space,is_tax_id_len_correct,len_acc_no,is_acc_no_len_correct,is_acc_contain_char,is_acc_contain_space,new_tax_id,new_acc_no,new_len_tax_id,new_len_acc_no,is_new_tax_id_len_correct,is_new_acc_no_len_correct,is_tax_id_null,is_acc_no_null,is_tax_id_pass_at_the_beginning,is_acc_no_pass_at_the_beginning,is_no_tier_correct
0,45ZWU8,NaN,NaN,[25.0],1.0,nan,3,1,0,1,3,1,1,0,,,0,0,1,1,0,1,1,1,0
1,45AAFR,NaN,NaN,[35.0],1.0,1100501097390,13,0,0,0,3,1,1,0,1100501097390,,13,0,0,1,0,1,0,1,0
2,45ZWJD,NaN,NaN,[20.0],1.0,nan,3,1,0,1,3,1,1,0,,,0,0,1,1,0,1,1,1,0
3,45AAFP,NaN,NaN,[20.0],1.0,3100101029938,13,0,0,0,3,1,1,0,3100101029938,,13,0,0,1,0,1,0,1,0
4,45ZWJH,NaN,NaN,[20.0],1.0,nan,3,1,0,1,3,1,1,0,,,0,0,1,1,0,1,1,1,0
5,45AAFY,kasikorn,4672314300,[30.0],1.0,1309902443791,13,0,0,0,10,0,0,0,1309902443791,4672314300,13,10,0,0,0,0,0,0,0
6,45ZWJT,NaN,NaN,[20.0],1.0,nan,3,1,0,1,3,1,1,0,,,0,0,1,1,0,1,1,1,0
7,45AAFF,KASIKORNBANK,263887719,[25.0],1.0,3 8499 00311 24 5,17,0,1,1,9,1,0,0,3849900311245,263887719,13,9,0,1,0,0,1,0,0
8,45ZW5T,NaN,NaN,[20.0],1.0,nan,3,1,0,1,3,1,1,0,,,0,0,1,1,0,1,1,1,0
9,45AAFC,SCB,4403024712,[30.0],1.0,3 5009 00023 13 9,17,0,1,1,10,0,0,0,3500900023139,4403024712,13,10,0,0,0,0,1,0,0


In [85]:
df_summary_selected_column["is_tax_id_pass_after_cleaning"]  = np.where(df_summary_selected_column["is_new_tax_id_len_correct"] == 0 , 0 ,1)
df_summary_selected_column["is_acc_no_pass_at_the_beginning"] = np.where(df_summary_selected_column["is_new_acc_no_len_correct"] == 0 , 0 ,1)
conditions_tax = [
    (df_summary_selected_column["is_tax_id_pass_at_the_beginning"] == 0 ),
    (df_summary_selected_column["is_new_tax_id_len_correct"] == 0),
    (df_summary_selected_column["is_tax_id_pass_at_the_beginning"] == 1 ) & (df_summary_selected_column["is_new_tax_id_len_correct"] == 1)
    ]
values = ['pass_at_the_beginning', 'pass_after_cleaning', 'not_pass']

conditions_acct_no = [
    (df_summary_selected_column["is_acc_no_pass_at_the_beginning"] == 0 ),
    (df_summary_selected_column["is_new_acc_no_len_correct"] == 0),
    (df_summary_selected_column["is_acc_no_pass_at_the_beginning"] == 1 ) & (df_summary_selected_column["is_new_acc_no_len_correct"] == 1)
    ]

df_summary_selected_column["tax_id_status"]  = np.select(conditions_tax, values)
df_summary_selected_column["acc_no_status"]  = np.select(conditions_acct_no, values)

Check Salesforce data formatting


In [86]:
df_col_value = pd.DataFrame(data = None,columns={"column_name","values","total_records"})


column_summary = ["is_tax_id_pass_at_the_beginning",	"is_acc_no_pass_at_the_beginning","is_tax_contain_char", "is_tax_contain_space", "is_tax_id_len_correct", "len_acc_no", "is_acc_no_len_correct", "is_acc_contain_char",' is_acc_contain_space']

col_len = len(column_summary)
for i in range(0, col_len-1) :
  c_name = column_summary[i]
  df_attribute = df_summary_selected_column.groupby(c_name).count()[['GRID']]
  df_attribute["column_name"] = column_summary[i]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'GRID': 'total_records'}, inplace = False )
  df_col_value = df_col_value.append(df_attribute)
print(df_col_value) 

   total_records                      column_name values
0         109033  is_tax_id_pass_at_the_beginning      0
1          14442  is_tax_id_pass_at_the_beginning      1
0          85288  is_acc_no_pass_at_the_beginning      0
1          38187  is_acc_no_pass_at_the_beginning      1
0         114973              is_tax_contain_char      0
1           8502              is_tax_contain_char      1
0         122843             is_tax_contain_space      0
1            632             is_tax_contain_space      1
0          74580            is_tax_id_len_correct      0
1          48895            is_tax_id_len_correct      1
0           1603                       len_acc_no      1
1          10461                       len_acc_no      3
2             13                       len_acc_no      4
3              3                       len_acc_no      6
4              2                       len_acc_no      7
5           2129                       len_acc_no      8
6          23976               

In [87]:
######## SUM value of acc_no / tax that how many errors points are exist in the column ########################
######## ERROR POINTS ARE FROM 1) incorrect of length 2) Containing of space 3) Containing of special char  ###
df_summary_selected_column["sum_account_checking"] = df_summary_selected_column["is_acc_contain_char"]+df_summary_selected_column["is_acc_contain_space"]+df_summary_selected_column["is_acc_no_len_correct"]
df_summary_selected_column["sum_tax_id_checking"] = df_summary_selected_column["is_tax_contain_char"]+df_summary_selected_column["is_tax_contain_space"]+df_summary_selected_column["is_tax_id_len_correct"]

######## SUM ERROR POINTS FOR BILLING TOOL #########
df_billing_new["sum_account_checking"] = df_billing_new["is_acc_contain_char"]+df_billing_new["is_acc_contain_space"]+df_billing_new["is_acc_no_len_correct"]
df_billing_new["sum_tax_id_checking"] = df_billing_new["is_tax_contain_char"]+df_billing_new["is_tax_contain_space"]+df_billing_new["is_tax_id_len_correct"]

####### COMBINE billing tool and salesforce #######
df_bill_sf_merge = pd.merge(df_summary_selected_column,df_billing_new , how='outer', left_on= 'GRID', right_on= 'salesforce_id', suffixes=(None, '_B'))



In [88]:
df_col_value_2 = pd.DataFrame(data = None,columns={"column_name","values","total_records"})

###### MATCH BANK ACCOUNT NAME ####################
df_bill_sf_merge = pd.merge(df_bill_sf_merge, df_bank_sf_unique , how = 'left', left_on = 'Bank Name', right_on = 'SF bank name' , suffixes = (None, '_SF_BANK_MASTER'))
df_bill_sf_merge = pd.merge(df_bill_sf_merge, df_bank_billing_unique, how = 'left', left_on = 'bank_name', right_on= 'BT bank name', suffixes = (None, '_BL_BANK_MASTER'))
df_bill_sf_merge = pd.merge(df_bill_sf_merge, df_bank_billing_unique, how = 'left', left_on = 'bank_name_swap', right_on= 'BT bank name', suffixes = (None, '_BL_BANK_SWAP'))

####### CALCULATE FINAL STATUS FOR THE VALIDATION #########
####### STATUS : 'OK' = PASS (ERROR FROM BOTH SIDE ARE 0), 'CHECK' = MATCH AND PASS ALL CRITERIA BUT NEED TO RECHECK
######           'NOT MATCH' = DONT PASS AT ALL , 'MISSING SF' = DATA MISSING FROM SF , 'MISSING BL' = DATA MISSING FROM BL
######           'MISSING BOTH" = DOES NOT HAVE THE DATA
df_bill_sf_merge["is_bank_acc_match_old"] = (df_bill_sf_merge["Bank Account Number"] ==  df_bill_sf_merge["bank_account_number"])
df_bill_sf_merge["is_tax_id_match_old"] = (df_bill_sf_merge["Tax Number"] ==  df_bill_sf_merge["tax_id"])
df_bill_sf_merge["is_bank_acc_match_new"] = (df_bill_sf_merge["new_acc_no"] ==  df_bill_sf_merge["new_acc_no_B"])
df_bill_sf_merge["is_tax_id_match_new"] = (df_bill_sf_merge["new_tax_id"] ==  df_bill_sf_merge["new_tax_id_B"])
df_bill_sf_merge["is_bank_acc_swap_match"] = (df_bill_sf_merge["Bank Account Number"] == df_bill_sf_merge["bank_account_number_swap_clean"])

con_acc_swap = [(df_bill_sf_merge["BT bank name_BL_BANK_SWAP"].astype(str) == df_bill_sf_merge["bank_name_swap"].astype(str)) & (df_bill_sf_merge["BT bank name_BL_BANK_SWAP"].astype(str) != 'No Bank Detail')]
value_acc_swap = [(1)]


df_bill_sf_merge["is_bank_name_swap_match"] = np.select(con_acc_swap,value_acc_swap)
df_bill_sf_merge["is_bank_name_swap_match"] = np.where(df_bill_sf_merge["is_bank_name_swap_match"] == 1,0,1)




In [89]:
##### ADD ZERO LEADING FOR TAX ID ######
def zero_lead (txt) :
  fill_txt = txt.zfill(13)
  return fill_txt

df_bill_sf_merge["new_tax_id_13_digit"] = df_bill_sf_merge["new_tax_id"].astype(str).apply(zero_lead)
df_bill_sf_merge["new_tax_id_B_13_digit"] =df_bill_sf_merge["new_tax_id_B"].astype(str).apply(zero_lead)


In [90]:
###### CALCULATE TAX MATCHING STATUS #####
conditions_tax = [
    ((df_bill_sf_merge["sum_tax_id_checking"] == 0 ) & (df_bill_sf_merge["Tax Number"] == df_bill_sf_merge["tax_id"] ) & (df_bill_sf_merge["sum_tax_id_checking_B"] == 0) & (df_bill_sf_merge["tax_id"].notnull()) & (df_bill_sf_merge["Tax Number"].notnull())),
    ((df_bill_sf_merge["is_new_tax_id_len_correct"] == 0 ) & (df_bill_sf_merge["new_tax_id"] == df_bill_sf_merge["new_tax_id_B"]  ) &(df_bill_sf_merge["is_new_tax_id_len_correct_B"] == 0) & (df_bill_sf_merge["tax_id"].notnull()) & (df_bill_sf_merge["Tax Number"].notnull())),
    ((df_bill_sf_merge["new_tax_id_B_13_digit"]  == df_bill_sf_merge["new_tax_id_13_digit"]) & ((df_bill_sf_merge["is_new_tax_id_len_correct_B"] == 0) | (df_bill_sf_merge["is_tax_id_len_correct"] == 0))),
    (((df_bill_sf_merge["is_new_tax_id_len_correct"] != 1  ) & (df_bill_sf_merge["new_tax_id_B"] != df_bill_sf_merge["new_tax_id"]) & (df_bill_sf_merge["new_tax_id_B"] != "") & (df_bill_sf_merge["new_tax_id"] != "")  &(df_bill_sf_merge["is_new_tax_id_len_correct_B"] != 1)) & (df_bill_sf_merge["tax_id"].notnull()) & (df_bill_sf_merge["Tax Number"].notnull())),
    ((df_bill_sf_merge["tax_id"].notnull()) & (df_bill_sf_merge["Tax Number"].isnull())),
    ((df_bill_sf_merge["tax_id"].isnull()) & (df_bill_sf_merge["Tax Number"].notnull())),
    ((df_bill_sf_merge["tax_id"].isnull()) & (df_bill_sf_merge["Tax Number"].isnull())),
    (( (df_bill_sf_merge["is_new_tax_id_len_correct"] == 0 ) & (df_bill_sf_merge["is_new_tax_id_len_correct_B"] == 0 ) )& (df_bill_sf_merge["new_tax_id"] == df_bill_sf_merge["new_tax_id_B"]  )),
    (((df_bill_sf_merge["new_tax_id"] == '0'  ) & (df_bill_sf_merge["new_tax_id_B"] == '0'  ))),
    (((df_bill_sf_merge["new_tax_id"] == '0') | (df_bill_sf_merge["new_tax_id"] == '') )),
    (((df_bill_sf_merge["new_tax_id_B"] == '0') | (df_bill_sf_merge["new_tax_id_B"] == '') )),
    ]

values = ['OK', 'CHECK','CHECK - ZERO LEADING','NOT MATCH', 'MISSING SF', 'MISSING BL', 'MISSING BOTH', "CHECK" , 'MISSING BOTH','MISSING SF', 'MISSING BL']

df_bill_sf_merge["status_tax_id"]  = np.select(conditions_tax, values)



In [91]:
chk = df_bill_sf_merge[["tax_id", "Tax Number","new_tax_id", "new_tax_id_B", "status_tax_id"]] 
chk[(chk["status_tax_id"]== "MISSING BOTH")]

,tax_id,Tax Number,new_tax_id,new_tax_id_B,status_tax_id
5843,#0,0,0,0,MISSING BOTH
20841,#0,0,0,0,MISSING BOTH
24441,#0,0,0,0,MISSING BOTH
46774,#0,0,0,0,MISSING BOTH
46808,#0,0,0,0,MISSING BOTH
48902,#0,0,0,0,MISSING BOTH
48904,#0,0,0,0,MISSING BOTH
48908,#0,0,0,0,MISSING BOTH
59410,#0,0,0,0,MISSING BOTH
59492,#0,0,0,0,MISSING BOTH


In [92]:
### ADD 0 LEADING FOR ACCOUNT NO ###

def zero_lead (txt) :
  fill_txt = txt.zfill(13)
  return fill_txt

df_bill_sf_merge["new_acc_no_13_digit"] = df_bill_sf_merge["new_acc_no"].astype(str).apply(zero_lead)
df_bill_sf_merge["new_acc_no_B_13_digit"] =df_bill_sf_merge["new_acc_no_B"].astype(str).apply(zero_lead)

In [93]:
#### CALCULATE STATUS_ACC_NO ###
df_bill_sf_merge["new_acc_no"] = df_bill_sf_merge["new_acc_no"].apply(cast_to_none)
df_bill_sf_merge["new_acc_no_B"] = df_bill_sf_merge["new_acc_no_B"].apply(cast_to_none)
df_bill_sf_merge["bank_account_number"] = df_bill_sf_merge["bank_account_number"].apply(cast_to_none)
df_bill_sf_merge["Bank Account Number"] = df_bill_sf_merge["Bank Account Number"].apply(cast_to_none)

conditions_acc = [
    ((df_bill_sf_merge["sum_account_checking"] == 0 ) & (df_bill_sf_merge["bank_account_number"] == df_bill_sf_merge["Bank Account Number"] ) & (df_bill_sf_merge["sum_account_checking_B"] == 0) & (df_bill_sf_merge["bank_account_number"].notnull()) & (df_bill_sf_merge["Bank Account Number"].notnull())),
    ((df_bill_sf_merge["is_new_acc_no_len_correct"] ==0 ) & (df_bill_sf_merge["new_acc_no"] == df_bill_sf_merge["new_acc_no_B"])  & (df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 0) & (df_bill_sf_merge["bank_account_number"].notnull()) & (df_bill_sf_merge["Bank Account Number"].notnull()) == True),
    ((df_bill_sf_merge["is_new_acc_no_len_correct"] ==0 ) & (df_bill_sf_merge["new_acc_no_B"] == df_bill_sf_merge["new_acc_no"] )  & (df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 0) & (df_bill_sf_merge["bank_account_number_swap_clean"] == df_bill_sf_merge["Bank Account Number"]) & (df_bill_sf_merge["bank_account_number_swap_clean"].notnull()) & (df_bill_sf_merge["Bank Account Number"].notnull())),
    (((df_bill_sf_merge["is_new_acc_no_len_correct"] == 1 ) | (df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 1)) & (df_bill_sf_merge["new_acc_no_B"] != df_bill_sf_merge["new_acc_no_B"]) & (df_bill_sf_merge["bank_account_number"].notnull()) & (df_bill_sf_merge["Bank Account Number"].notnull())),
    (((df_bill_sf_merge["bank_account_number"].isnull()) |(df_bill_sf_merge["bank_account_number"] == '')) & ((df_bill_sf_merge["Bank Account Number"].notnull()) |  (df_bill_sf_merge["Bank Account Number"] != '')) & (df_bill_sf_merge["is_acc_no_len_correct"] == 0 )),
    (((df_bill_sf_merge["bank_account_number"].notnull()) | (df_bill_sf_merge["bank_account_number"] != '')) & ((df_bill_sf_merge["Bank Account Number"].isnull()) | (df_bill_sf_merge["Bank Account Number"] == '' )) & (df_bill_sf_merge["is_acc_no_len_correct_B"] == 0 )),
    ((
      (((df_bill_sf_merge["bank_account_number"].isnull()) & (df_bill_sf_merge["new_acc_no"].isnull())) &
      ((df_bill_sf_merge["Bank Account Number"].isnull()) & (df_bill_sf_merge["new_acc_no_B"].isnull()))))),
    ((df_bill_sf_merge["new_acc_no_B_13_digit"]  == df_bill_sf_merge["new_acc_no_13_digit"]) & ((df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 0) | (df_bill_sf_merge["is_new_acc_no_len_correct"] == 0))),
    ((df_bill_sf_merge["is_new_acc_no_len_correct"] == 0) & (df_bill_sf_merge["new_acc_no_B"] == "")),
    ((df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 0) & (df_bill_sf_merge["new_acc_no"] == "")),
    ((df_bill_sf_merge["is_new_acc_no_len_correct_B"] == 0) & ((df_bill_sf_merge["new_acc_no"] == "") & (df_bill_sf_merge["new_acc_no_B"] == ""))),
    (((df_bill_sf_merge["new_acc_no"] == df_bill_sf_merge["bank_account_number_swap_clean"]) & (df_bill_sf_merge["is_new_acc_no_len_correct"] == 0 )& (df_bill_sf_merge["bank_account_number_swap_clean"].notnull()) )),
    ((df_bill_sf_merge["new_acc_no"] != df_bill_sf_merge["new_acc_no_B"] ))
    ]

values_acc = ['OK', 'CHECK', 'CHECK','NOT MATCH', 'MISSING BL', 'MISSING SF',  'MISSING BOTH','CHECK - ZERO LEADING', 'MISSING SF', 'MISSING BL', "MISSING BOTH" ,'CHECK - SWAP BANK ACC NO WITH BANK NAME','NOT MATCH']

df_bill_sf_merge["status_acc_no"] = np.select(conditions_acc,values_acc)

In [94]:
#### CLEAN BANK NAME MATCHING ####
def strip_NA(dt):
  if dt == '#N/A' : 
    return np.nan
  return dt

df_bill_sf_merge["name_for_upload_sf"] = df_bill_sf_merge["name_for_upload_sf"].apply(strip_NA)
df_bill_sf_merge["name_for_upload_sf"] = df_bill_sf_merge["name_for_upload_sf"].apply(cast_to_none)
df_bill_sf_merge["name_for_upload_B"] = df_bill_sf_merge["name_for_upload_B"].apply(strip_NA)
df_bill_sf_merge["name_for_upload_B"] = df_bill_sf_merge["name_for_upload_B"].apply(cast_to_none)


In [95]:
####### BANK ACCOUNT MASTER MATCHING STATUS ######
df_bill_sf_merge.rename(columns={"BT bank name_BL_BANK_SWAP":"BT_bank_name_BL_BANK_SWAP" }, inplace=True
                        )
conditions_acc_name = [
                        (df_bill_sf_merge["name_for_upload_sf"].notnull() & df_bill_sf_merge["name_for_upload_B"].notnull() & (df_bill_sf_merge["name_for_upload_sf"] == df_bill_sf_merge["name_for_upload_B"])),
                        ((df_bill_sf_merge["is_bank_name_swap_match"] == '0') & ((df_bill_sf_merge["BT_bank_name_BL_BANK_SWAP"].notnull()) | (df_bill_sf_merge["BT_bank_name_BL_BANK_SWAP"] != 'No Bank Detail') | (df_bill_sf_merge["BT_bank_name_BL_BANK_SWAP"] != 'check') | (df_bill_sf_merge["name_for_upload_B_BL_BANK_SWAP"] != 'check') | (df_bill_sf_merge["name_for_upload_B_BL_BANK_SWAP"] != '' ) | (df_bill_sf_merge["name_for_upload_B_BL_BANK_SWAP"].notnull()))),
                        (df_bill_sf_merge["name_for_upload_sf"].notnull() & df_bill_sf_merge["name_for_upload_B"].notnull() & (df_bill_sf_merge["name_for_upload_sf"] != df_bill_sf_merge["name_for_upload_B"])),
                        (df_bill_sf_merge["name_for_upload_sf"].isnull() & df_bill_sf_merge["name_for_upload_B"].notnull()),
                        (df_bill_sf_merge["name_for_upload_sf"].notnull() & df_bill_sf_merge["name_for_upload_B"].isnull() & (df_bill_sf_merge["name_for_upload_sf"].notnull() & df_bill_sf_merge["name_for_upload_B"].isnull())),
                        (df_bill_sf_merge["name_for_upload_sf"].isnull() & df_bill_sf_merge["name_for_upload_B"].isnull() & df_bill_sf_merge["Bank Name"].notnull() & df_bill_sf_merge["bank_name"].notnull()),
                        (df_bill_sf_merge["name_for_upload_sf"].isnull() & df_bill_sf_merge["name_for_upload_B"].isnull())
                       ]

####### ADD BANK NAME CONDITION ########
values_acc_name = [
                    'OK', 
                    'CHECK - BANK NAME SWAP WITH BANK ACC NO',
                    'NOT MATCH' ,
                    'MISSING FROM SF BANK MASTER', 
                    'MISSING FROM BL BANK MASTER',
                    'MISSING BOTH - BUT BOTH SF BL HAVE BANK NAME', 
                    'MISSING BOTH'
                    ]
df_bill_sf_merge["status_account_master_matching"]  = np.select(conditions_acc_name, values_acc_name)


column_summary_2 = ["is_bank_acc_match_old",	"is_tax_id_match_old","is_bank_acc_match_new", "is_tax_id_match_new",
                    "is_tax_id_pass_at_the_beginning",	"is_acc_no_pass_at_the_beginning","is_tax_contain_char", "is_tax_contain_space", 
                    "is_tax_id_len_correct", "is_acc_no_len_correct", "is_acc_contain_char","is_acc_contain_space","len_acc_no","new_len_acc_no","len_tax_id","new_len_tax_id","is_bank_acc_swap_match"]

col_len_2 = len(column_summary_2)
for sf in range(0, col_len_2) :
  c_name = column_summary_2[sf]
  df_attribute = df_bill_sf_merge.groupby(c_name).count()[['GRID']]
  df_attribute["column_name"] = column_summary_2[sf]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'GRID': 'total_records'}, inplace = False )
  df_col_value_2 = df_col_value_2.append(df_attribute)
print(df_col_value_2) 

   total_records             column_name values
0         123400   is_bank_acc_match_old  False
1             75   is_bank_acc_match_old   True
0         117827     is_tax_id_match_old  False
1           5648     is_tax_id_match_old   True
0          47641   is_bank_acc_match_new  False
..           ...                     ...    ...
16             1          new_len_tax_id     19
17             1          new_len_tax_id     26
18             1          new_len_tax_id     27
0         110428  is_bank_acc_swap_match  False
1          13047  is_bank_acc_swap_match   True

[88 rows x 3 columns]


In [96]:

## CLEAN TIERS ###
def clean_tier (x):
  if x == 0 :
    return None
  elif x == '' :
    return None
  else :
    return x

#### FORMAT TIERS ####

df_bill_sf_merge["total_tiers"] = df_bill_sf_merge["total_tiers"].apply(clean_tier)
df_bill_sf_merge["total_tiers_b"] = df_bill_sf_merge["total_tiers_b"].apply(clean_tier)

def compare_array(a,b) :
  a = str(a)
  b = str(b)
  dif = np.array_equal(a,b)
  if dif == True :
    return 0
  else : 
    return 1

df_bill_sf_merge["is_all_tier_match"] = np.vectorize(compare_array)(df_bill_sf_merge['Commission_tier'], df_bill_sf_merge['stv'])




In [97]:
df_bill_sf_merge["is_all_tier_match"]

0         1
1         0
2         1
3         0
4         1
         ..
152717    1
152718    1
152719    1
152720    1
152721    1
Name: is_all_tier_match, Length: 152722, dtype: int64

In [98]:
##CALCULATE COMMISSION STATUS ##
conditions_commission = [
                       ((df_bill_sf_merge["total_tiers"] > 5) & (df_bill_sf_merge["total_tiers_b"] > 5 )),
                       (df_bill_sf_merge["total_tiers"] > 5) ,
                       (df_bill_sf_merge["total_tiers_b"] > 5 ),
                       ((df_bill_sf_merge["total_tiers"].isnull()) & (df_bill_sf_merge["total_tiers_b"].isnull())   & (df_bill_sf_merge["stv"].isnull()))  ,
                       (df_bill_sf_merge["total_tiers"].isnull()) ,
                       (df_bill_sf_merge["total_tiers_b"].isnull()),
                       ((df_bill_sf_merge["is_all_tier_match"] == 0 ) & (df_bill_sf_merge["total_tiers_b"] > 1 )),
                       ((df_bill_sf_merge["is_all_tier_match"] == 0 ) & (df_bill_sf_merge["total_tiers_b"] == 1 ))
                       ]

####### ADD BANK NAME CONDITION ########
values_commission = [
                    'MISMATCH - NO.TIERS (BOTH) > 5',
                    'MISMATCH - NO.TIERS (SF) > 5', 
                    'MISMATCH - NO.TIERS (BL) > 5',
                    'MISSING - BOTH SF AND BL',
                    'MISSING FROM SF', 
                    'MISSING FROM BL',
                    'MATCH - BOTH SIDE HAVE THE SAME TIERS VALUES (> 1 TIERS)',
                    'MATCH - STANDARD COMMISSION'
                    ]
df_bill_sf_merge["status_commission"]  = np.select(conditions_commission, values_commission)

In [99]:
chk = df_bill_sf_merge[["status_commission", "total_tiers", "total_tiers_b", "stv", "Commission_tier"]] 
chk[((chk["status_commission"] =='0') )]
#chk[(chk["total_tiers_b"] > 1)]

,status_commission,total_tiers,total_tiers_b,stv,Commission_tier
44,0,2.0,1.0,[30.0],"[27.0, 30.0]"
50,0,1.0,1.0,[30.0],[27.0]
60,0,1.0,1.0,[30.0],[27.0]
62,0,3.0,1.0,[27.0],"[25.0, 27.0, 30.0]"
64,0,2.0,1.0,[30.0],"[27.0, 30.0]"
...,...,...,...,...,...
120106,0,2.0,1.0,[27.0],"[27.0, 30.0]"
122012,0,2.0,1.0,[12.0],"[12.0, 15.0]"
122088,0,2.0,1.0,[12.0],"[12.0, 15.0]"
122927,0,2.0,1.0,[10.0],"[10.0, 15.0]"


In [100]:
df_bill_sf_merge[(df_bill_sf_merge["status_account_master_matching"] == 'CHECK - BANK NAME SWAP WITH BANK ACC NO')]
#df_bill_sf_merge["is_no_tiers_match"] = (df_bill_sf_merge["total_tier"] == df_bill_sf_merge["number_tiers"])
#df_bill_sf_merge["is_commission_match"] = np.in1d(df_bill_sf_merge["stv"],df_bill_sf_merge["Commission Tier"])

,GRID,Bank Name,Bank Account Number,Commission_tier,total_tiers,Tax Number,len_tax_id,is_tax_contain_char,is_tax_contain_space,is_tax_id_len_correct,len_acc_no,is_acc_no_len_correct,is_acc_contain_char,is_acc_contain_space,new_tax_id,new_acc_no,new_len_tax_id,new_len_acc_no,is_new_tax_id_len_correct,is_new_acc_no_len_correct,is_tax_id_null,is_acc_no_null,is_tax_id_pass_at_the_beginning,is_acc_no_pass_at_the_beginning,is_no_tier_correct,is_tax_id_pass_after_cleaning,tax_id_status,acc_no_status,sum_account_checking,sum_tax_id_checking,client_name,tax_id,bank_account_number,bank_name,vendor_code,vendor_name,salesforce_id,sf_account_id,stv,total_tiers_b,...,is_sf_account_id_null,is_tax_id_pass_at_the_begginning,is_acc_no_pass_at_the_begginning,bank_account_number_swap_clean,len_acc_no_swap,is_len_acc_no_swap_correct,is_teir_excceed_5,is_tax_id_pass_after_cleaning_B,is_acc_no_pass_after_cleaning,tax_id_status_B,acc_no_status_B,sum_account_checking_B,sum_tax_id_checking_B,SF bank name,clean_name_sf,chk_sf,name_for_upload_sf,BT bank name,clean_name_B,chk_B,name_for_upload_B,BT_bank_name_BL_BANK_SWAP,clean_name_B_BL_BANK_SWAP,chk_B_BL_BANK_SWAP,name_for_upload_B_BL_BANK_SWAP,is_bank_acc_match_old,is_tax_id_match_old,is_bank_acc_match_new,is_tax_id_match_new,is_bank_acc_swap_match,is_bank_name_swap_match,new_tax_id_13_digit,new_tax_id_B_13_digit,status_tax_id,new_acc_no_13_digit,new_acc_no_B_13_digit,status_acc_no,status_account_master_matching,is_all_tier_match,status_commission


**SALESFORCE SUMMARY**


In [101]:
#1) TOTAL RECORDS
#2) NUMBER OF DUPLICATE RECORD FOR BANK ACCOUNT NUMBER / TAX ID 
#3) TOTAL MISSING VALUE FOR BANK ACCOUNT/ TAX ID 
#4) TOTAL COMMISSION VENDER BY COMMISSION TIER
#5) ALL FLAG LIST
#6) SUM IS_...
print("------------------------------SALESFORCE SUMMARY-----------------------------------------")
print("1) TOTAL RECORDS FOR BANK NAME FILE : " + str(len(df_bankname)))
print("   TOTAL DUPLICATE FOR BANK FILE")
print("   GRID : " + str(df_bankname["GRID"].duplicated().sum()))
print("   Bank Detail: ID : " + str(df_bankname["Bank Detail: ID"].duplicated().sum()))
print("   Bank Account Number : " + str(df_bankname["Bank Detail: ID"].duplicated().sum()))
print("   TOTAL UNIQUE BANK NAME :" + str(len(df_bankname.groupby("Bank Name").nunique())))

print(" 2) TOTAL RECORDS FOR TAX FILE : " + str(len(df_tax)))
print("   TOTAL DUPLICATE FOR TAX FILE")
print("   GRID : " + str(df_tax["GRID"].duplicated().sum()))
print("   Account ID : " + str(df_tax["Account ID"].duplicated().sum()))
print("   Tax Number : " + str(df_tax["Tax Number"].duplicated().sum()))

print("3) TOTAL RECORD FOR COMMISION FILE : " +str(len(df_commission)))
print("   TOTAL DUPLICATE FOR GRID : " + str(df_commission["GRID"].duplicated().sum()))
print("   TOTAL UNIQUE GRID : " + str(df_commission["GRID"].nunique()))
print("   TOTAL VENDOR WITHOUT TIER : " + str(df_comm.list_commission_tier.isnull().sum()))
print("   TOTAL VENDOR WITH TIER : " + str(df_comm.list_commission_tier.notnull().sum())) 
print(df_comm.groupby("total_tiers").count()[['GRID']])
#### PIVOT LIST ######
print("4) TOTAL RECORDS AFTER JOINING : " +str(len(df_summary_selected_column["GRID"])))

df_col_sf= pd.DataFrame(data = None,columns={"column_name","values","total_records"})
column_sf = ["is_tax_id_pass_at_the_beginning",	"is_acc_no_pass_at_the_beginning","is_tax_contain_char", "is_tax_contain_space", "is_tax_id_len_correct",  "is_acc_no_len_correct", 
             "is_acc_contain_char",'is_acc_contain_space']
col_sf = len(column_sf)
for i in range(0, col_sf) :
  c_name = column_sf[i]
  df_attribute = df_summary_selected_column.groupby(c_name).count()[['GRID']]
  df_attribute["column_name"] = column_sf[i]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'GRID': 'total_records'}, inplace = False )
  df_col_sf = df_col_sf.append(df_attribute)
#print(df_col_sf) 

print(pd.pivot_table(df_col_sf, values=['total_records'], index=None,
                    columns=['column_name',"values"], aggfunc=np.sum))

print("5) TOTAL VENDORS BY NUMBER OF NOT PASS ACCOUNT NO. CHECKING CRITERIA ")
df_summary_selected_column["sum_account_checking"] = df_summary_selected_column["is_acc_contain_char"]+df_summary_selected_column["is_acc_contain_space"]+df_summary_selected_column["is_acc_no_len_correct"]
print(df_summary_selected_column.groupby("sum_account_checking").count()[['GRID']])

print("6) TOTAL VENDORS BY NUMBER OF NOT PASS TAX_ID. CHECKING CRITERIA ")
df_summary_selected_column["sum_tax_id_checking"] = df_summary_selected_column["is_tax_contain_char"]+df_summary_selected_column["is_tax_contain_space"]+df_summary_selected_column["is_tax_id_len_correct"]
print(df_summary_selected_column.groupby("sum_tax_id_checking").count()[['GRID']])
print("TOTAL RECORDS NEED TO BE CLEANED UP :")
print(str(len(df_summary_selected_column[((df_summary_selected_column["sum_tax_id_checking"] != 0) | (df_summary_selected_column["sum_account_checking"] != 0) )].index)))

------------------------------SALESFORCE SUMMARY-----------------------------------------
1) TOTAL RECORDS FOR BANK NAME FILE : 115538
   TOTAL DUPLICATE FOR BANK FILE
   GRID : 877
   Bank Detail: ID : 0
   Bank Account Number : 0
   TOTAL UNIQUE BANK NAME :1502
 2) TOTAL RECORDS FOR TAX FILE : 123335
   TOTAL DUPLICATE FOR TAX FILE
   GRID : 0
   Account ID : 0
   Tax Number : 54555
3) TOTAL RECORD FOR COMMISION FILE : 123767
   TOTAL DUPLICATE FOR GRID : 583
   TOTAL UNIQUE GRID : 123184
   TOTAL VENDOR WITHOUT TIER : 123547
   TOTAL VENDOR WITH TIER : 802
             GRID
total_tiers      
1.0             1
2.0            96
3.0           225
5.0           480
4) TOTAL RECORDS AFTER JOINING : 123475
column_name   is_acc_contain_char         ... is_tax_id_pass_at_the_beginning       
values                          0      1  ...                               0      1
total_records              113000  10475  ...                          109033  14442

[1 rows x 15 columns]
5) TOTAL

In [102]:
!pip install xlsxwriter 
import pandas as pd
import numpy as np

# Sample DataFrame
writer = pd.ExcelWriter('SF_MATCHING.xlsx', engine='xlsxwriter')
df = pd.DataFrame(data = None)
book = writer.book
header_format = book.add_format({'bold': True, 'size': 12})
body_format = book.add_format({'bold': False, 'size': 8})


# Get book and sheet objects for futher manipulation below

df.to_excel(writer, sheet_name = 'Salesforce' , startrow = 1 )
sheet = writer.sheets['Salesforce']

sheet.write('A1', 'Salesforce Data Quality', header_format)
sheet.write('A3', '1) TOTAL RECORDS FOR BANK NAME FILE  ',body_format) 
sheet.write('B3', len(df_bankname),body_format) 

sheet.write('A4', 'TOTAL DUPLICATE FOR BANK FILE : ',body_format) 
sheet.write('A5', 'GRID  : ',body_format) 
sheet.write('B5', df_bankname["GRID"].duplicated().sum(),body_format)

sheet.write('A6', 'Bank Detail: ID : ',body_format) 
sheet.write('B6',  df_bankname["Bank Detail: ID"].duplicated().sum(),body_format)
sheet.write('A7', 'Bank Account Number : ',body_format) 
sheet.write('B7' , df_bankname["Bank Detail: ID"].duplicated().sum() ,body_format)
sheet.write('D7',  "TOTAL UNIQUE BANK NAME :" , body_format)
sheet.write('E7',  len(df_bankname.groupby("Bank Name").nunique()), body_format)

sheet.write('A8', "2) TOTAL RECORD FOR TAX FILE : " , body_format) 
sheet.write('B8', len(df_tax),body_format)
sheet.write('A9', 'TOTAL DUPLICATE FOR TAX FILE',body_format) 
sheet.write('A10', 'GRID : ',body_format) 
sheet.write('B10', df_tax["GRID"].duplicated().sum(),body_format)
sheet.write('A11', ' Account ID : ',body_format) 
sheet.write('B11',  df_tax["Account ID"].duplicated().sum(),body_format)
sheet.write('A12', 'Tax Number :',body_format)
sheet.write('B12', df_tax["Tax Number"].duplicated().sum(),body_format)
sheet.write('A14', '3) TOTAL RECORD FOR COMMISION FILE : ',body_format) 
sheet.write('B14', len(df_commission),body_format)
sheet.write('A15', 'TOTAL DUPLICATE FOR GRID :',body_format)  
sheet.write('B15', df_commission["GRID"].duplicated().sum(),body_format)  
sheet.write('A16','TOTAL UNIQUE GRID : ',body_format)
sheet.write('B16',df_commission["GRID"].nunique(),body_format)
sheet.write('A17','TOTAL VENDOR WITHOUT TIER :' ,body_format)
sheet.write('B17',df_comm.list_commission_tier.isnull().sum(),body_format)
sheet.write('D17', 'TOTAL UNIQUE VENDORS WITHOUT TIER : ')

##df_comm.list_commission_tier.isnull().duplicated().sum()
select_no_tier = df_commission[(df_commission["Tier ID"].isnull())]
#### TOTAL RECORD WITHOUT TIER
print(len(select_no_tier["GRID"].index))
sheet.write('E17', (select_no_tier["GRID"].nunique()), body_format )

sheet.write('A18',"TOTAL VENDOR WITH TIER : ",body_format)
sheet.write('B18', df_comm.list_commission_tier.notnull().sum(),body_format)
total_grid_by_tier = (df_comm.groupby("total_tiers").count()[['GRID']])
total_grid_by_tier.to_excel(writer, sheet_name = 'Salesforce' , startrow = 20 )

sheet.write('A30',"4) TOTAL RECORDS AFTER JOINING : " ,body_format )
sheet.write('B30' ,len(df_summary_selected_column["GRID"]), body_format)

df_col_sf= pd.DataFrame(data = None,columns={"column_name","values","total_records"})
column_sf = ["is_tax_id_pass_at_the_beginning",	"is_acc_no_pass_at_the_beginning","is_tax_contain_char", "is_tax_contain_space", "is_tax_id_len_correct",  "is_acc_no_len_correct", 
             "is_acc_contain_char",'is_acc_contain_space']
col_sf = len(column_sf)
for i in range(0, col_sf) :
  c_name = column_sf[i]
  df_attribute = df_summary_selected_column.groupby(c_name).count()[['GRID']]
  df_attribute["column_name"] = column_sf[i]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'GRID': 'total_records'}, inplace = False )
  df_col_sf = df_col_sf.append(df_attribute)
#print(df_col_sf) 

pivot_check = (pd.pivot_table(df_col_sf, values=['total_records'], index=None,
                    columns=['column_name',"values"], aggfunc=np.sum))

pivot_check.to_excel(writer, sheet_name = 'Salesforce' , startrow = 33 )

sheet.write('A40',"5) TOTAL VENDORS BY NUMBER OF NOT PASS ACCOUNT NO. CHECKING CRITERIA :" , body_format)

df_summary_selected_column["sum_account_checking"] = df_summary_selected_column["is_acc_contain_char"]+df_summary_selected_column["is_acc_contain_space"]+df_summary_selected_column["is_acc_no_len_correct"]
sum_account_checking = (df_summary_selected_column.groupby("sum_account_checking").count()[['GRID']])
sum_account_checking.to_excel(writer, sheet_name = 'Salesforce', startrow = 42)

sheet.write('A50' , "6) TOTAL VENDORS BY NUMBER OF NOT PASS TAX_ID. CHECKING CRITERIA " , body_format)


df_summary_selected_column["sum_tax_id_checking"] = df_summary_selected_column["is_tax_contain_char"]+df_summary_selected_column["is_tax_contain_space"]+df_summary_selected_column["is_tax_id_len_correct"]
sum_tax_checking = (df_summary_selected_column.groupby("sum_tax_id_checking").count()[['GRID']])
sum_tax_checking.to_excel(writer, sheet_name = 'Salesforce' , startrow = 52)

sheet.write("A60", "7) TOTAL RECORDS NEED TO BE CLEANED (TAX_ID OR ACC_NO ANY NEED TO BE CLEANED) : " , body_format)
sheet.write("B60", str(len(df_summary_selected_column[((df_summary_selected_column["sum_tax_id_checking"] != 0) | (df_summary_selected_column["sum_account_checking"] != 0) )].index)),body_format)

sheet.write ("A65", "8) DISTRIBUTION OF COMMISSION TIER : ")
pv_tier = df_commission.groupby("Tier Name").count()[['GRID']]
pv_tier.to_excel(writer, sheet_name = "Salesforce", startrow= 67)  



123548


**BILLING TOOL SUMMARY**

In [103]:
#1) TOTAL RECORDS
#2) NUMBER OF DUPLICATE RECORD FOR BANK ACCOUNT NUMBER / TAX ID 
#3) TOTAL MISSING VALUE FOR BANK ACCOUNT/ TAX ID 
#4) TOTAL COMMISSION VENDER BY COMMISSION TIER
#5) ALL FLAG LIST
#6) SUM IS_...
print("------------------------------BILLING TOOL SUMMARY-----------------------------------------")
print("1) TOTAL RECORDS  : " + str(len(df_billing)))
print("2) TOTAL VENDOR_CODE :" + str(len(df_billing[(df_billing["vendor_code"].notnull())].index)))
print("3) TOTAL CHAIN STORES VENDORS : " + str(len(df_billing[(df_billing["chain_code"].notnull())].index)))
print("4) TOTAL UNIQUE CHAIN CODE : " + str(len(df_billing[(df_billing["chain_code"].notnull())].nunique())))
print("   TOTAL DUPLICATE")
print("   vendor_code : " + str(df_billing["vendor_code"].duplicated().sum()))
print("   chain_code : " + str((df_billing["chain_code"] != None).notnull().duplicated().sum()))
print("   salesforce_id : " + str(df_billing["salesforce_id"].duplicated().sum()))
print("   tax_id : " + str(df_billing["tax_id"].duplicated().sum()))
print("   bank_account_number : " + str(df_billing["bank_account_number"].duplicated().sum()))
print("   TOTAL NULL")
print("   vendor_code : " + str(df_billing["vendor_code"].isnull().sum()))
print("   salesforce_id : " + str(df_billing["salesforce_id"].isnull().sum()))
print("   tax_id : " + str(df_billing["tax_id"].isnull().sum()))
print("   bank_account_number : " + str(df_billing["bank_account_number"].isnull().sum()))
print("5) COMMISSION TIER : ")
print(df_billing.groupby("total_tiers_b").count()[['vendor_code']])

df_col_bl= pd.DataFrame(data = None,columns={"column_name","values","total_records"})
column_bl = ["is_tax_id_pass_at_the_begginning",	"is_acc_no_pass_at_the_begginning","is_tax_contain_char", "is_tax_contain_space", "is_tax_id_len_correct",  "is_acc_no_len_correct", 
             "is_acc_contain_char",'is_acc_contain_space']
col_bl = len(column_bl)
for i in range(0, col_bl) :
  c_name = column_bl[i]
  df_attribute = df_billing_new.groupby(c_name).count()[['vendor_code']]
  df_attribute["column_name"] = column_sf[i]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'vendor_code': 'total_records'}, inplace = False )
  df_col_bl = df_col_bl.append(df_attribute)
#print(df_col_sf) 
with pd.option_context('display.max_rows', 20, 'display.max_columns', 20):  
    print(pd.pivot_table(df_col_bl, values=['total_records'], index=None,
                    columns=['column_name',"values"], aggfunc=np.sum))

print("5) TOTAL VENDORS BY NUMBER OF NOT PASS ACCOUNT NO. CHECKING CRITERIA ")
df_billing_new["sum_account_checking"] = df_billing_new["is_acc_contain_char"]+df_billing_new["is_acc_contain_space"]+df_billing_new["is_acc_no_len_correct"]
print(df_billing_new.groupby("sum_account_checking").count()[['vendor_code']])

print("6) TOTAL VENDORS BY NUMBER OF NOT PASS TAX_ID. CHECKING CRITERIA ")
df_billing_new["sum_tax_id_checking"] = df_billing_new["is_tax_contain_char"]+df_billing_new["is_tax_contain_space"]+df_billing_new["is_tax_id_len_correct"]
print(df_billing_new.groupby("sum_tax_id_checking").count()[['vendor_code']])
print("7) TOTAL RECORDS NEED TO BE CLEANED (both account_no and tax_id not pass at the beginning ) : ")
print(str(len(df_billing_new[((df_billing_new["sum_tax_id_checking"] != 0) | (df_billing_new["sum_account_checking"] != 0) )].index)))




------------------------------BILLING TOOL SUMMARY-----------------------------------------
1) TOTAL RECORDS  : 150382
2) TOTAL VENDOR_CODE :150382
3) TOTAL CHAIN STORES VENDORS : 10371
4) TOTAL UNIQUE CHAIN CODE : 18
   TOTAL DUPLICATE
   vendor_code : 0
   chain_code : 150381
   salesforce_id : 179
   tax_id : 26651
   bank_account_number : 43987
   TOTAL NULL
   vendor_code : 0
   salesforce_id : 180
   tax_id : 0
   bank_account_number : 5
5) COMMISSION TIER : 
               vendor_code
total_tiers_b             
1                   150344
2                        4
3                       20
5                       14
column_name   is_acc_contain_char         is_acc_contain_space         \
values                          0       1                    0      1   
total_records                1030  149352               122905  27477   

column_name   is_acc_no_len_correct        is_acc_no_pass_at_the_beginning  \
values                            0      1                            

**PRINT BILLING TOOL SUMMARY TO EXCEL**


In [104]:
df2 = pd.DataFrame(data = None)

# Get book and sheet objects for futher manipulation below
#book = writer.book
df2.to_excel(writer, sheet_name = 'Billing_Tool' , startrow = 1 )
sheet2 = writer.sheets['Billing_Tool']

sheet2.write('A1', 'Billing Tool Data Quality', header_format)


sheet2.write('A2',"1) TOTAL RECORDS  : ",body_format)
sheet2.write('B2',(len(df_billing)))
sheet2.write('A3',"2) TOTAL CHAIN STORES VENDORS : ", body_format)
sheet2.write('B3',len(df_billing[(df_billing["chain_code"].notnull())].index), body_format)
sheet2.write('A4', "3) TOTAL UNIQUE CHAIN CODE : ", body_format)
sheet2.write('B4',len(df_billing[(df_billing["chain_code"].notnull())].nunique()), body_format)

sheet2.write('A5',"TOTAL DUPLICATE" , body_format)
sheet2.write('A6',"vendor_code : " , body_format)
sheet2.write('B6' ,(df_billing["vendor_code"].duplicated().sum()), body_format)
sheet2.write("A7", "salesforce_id : " , body_format)
sheet2.write('B7', (df_billing["salesforce_id"].duplicated().sum()), body_format)
sheet2.write('A8', "tax_id : ", body_format)
sheet2.write('B8', df_billing["tax_id"].duplicated().sum(), body_format)
sheet2.write('A9', "bank_account_number : ", body_format)
sheet2.write('B9',(df_billing["bank_account_number"].duplicated().sum()), body_format)
sheet2.write('A10' ,"TOTAL NULL", body_format)
sheet2.write('A11',"vendor_code : " , body_format)
sheet2.write('B11',(df_billing["vendor_code"].isnull().sum()), body_format)
sheet2.write('A12' , "salesforce_id : ", body_format)
sheet2.write('B12', (df_billing["salesforce_id"].isnull().sum()), body_format)
sheet2.write('A13',"tax_id : " , body_format)
sheet2.write('B13', (df_billing["tax_id"].isnull().sum()), body_format)
sheet2.write('A14', "bank_account_number : ", body_format)
sheet2.write('B14', (df_billing["bank_account_number"].isnull().sum()),body_format)
sheet2.write('A15', "5) COMMISSION TIERS :")
pv_commission_tier = pd.DataFrame(df_billing.groupby("total_tiers_b").count()[['vendor_code']])
pv_commission_tier.to_excel(writer, sheet_name = 'Billing_Tool', startrow =17)

df_col_bl= pd.DataFrame(data = None,columns={"column_name","values","total_records"})
column_bl = ["is_tax_id_pass_at_the_begginning",	"is_acc_no_pass_at_the_begginning","is_tax_contain_char", "is_tax_contain_space", "is_tax_id_len_correct",  "is_acc_no_len_correct", 
             "is_acc_contain_char",'is_acc_contain_space']
col_bl = len(column_bl)
for i in range(0, col_bl) :
  c_name = column_bl[i]
  df_attribute = df_billing_new.groupby(c_name).count()[['vendor_code']]
  df_attribute["column_name"] = column_sf[i]
  df_attribute = df_attribute.reset_index()
  df_attribute = df_attribute.rename(columns = {c_name: 'values', 'vendor_code': 'total_records'}, inplace = False )
  df_col_bl = df_col_bl.append(df_attribute)
#print(df_col_sf) 
pv_total_record_billing =   pd.pivot_table(df_col_bl, values=['total_records'], index=None,
                                                  columns=['column_name',"values"], aggfunc=np.sum)


pv_total_record_billing.to_excel(writer, sheet_name = 'Billing_Tool' , startrow = 27)
sheet2.write('A34', "2) TOTAL VENDORS BY NUMBER OF NOT PASS ACCOUNT NO. CHECKING CRITERIA ", body_format)

df_billing_new["sum_account_checking"] = df_billing_new["is_acc_contain_char"]+df_billing_new["is_acc_contain_space"]+df_billing_new["is_acc_no_len_correct"]
pv_acct_check = (df_billing_new.groupby("sum_account_checking").count()[['vendor_code']])

pv_acct_check.to_excel(writer, sheet_name = 'Billing_Tool', startrow =37)

sheet2.write('A45', "3) TOTAL VENDORS BY NUMBER OF NOT PASS TAX_ID. CHECKING CRITERIA ", body_format)
df_billing_new["sum_tax_id_checking"] = df_billing_new["is_tax_contain_char"]+df_billing_new["is_tax_contain_space"]+df_billing_new["is_tax_id_len_correct"]
pv_tax_check = (df_billing_new.groupby("sum_tax_id_checking").count()[['vendor_code']])
pv_tax_check.to_excel(writer, sheet_name = 'Billing_Tool', startrow = 47)

sheet2.write("A58", "4) TOTOL RECORDS NEED TO BE CLEANED (ACCOUNT_NO OR TAX_ID are incorrect):  " , body_format)
sheet2.write("B58", (len(df_billing_new[((df_billing_new["sum_tax_id_checking"] != 0) | (df_billing_new["sum_account_checking"] != 0) )].index)), body_format)

#writer.save()


0

In [105]:
df4 = pd.DataFrame(data = None)


# Get book and sheet objects for futher manipulation below
#book = writer.book
df4.to_excel(writer, sheet_name = 'Bank Master Checking' , startrow = 1 )
sheet4 = writer.sheets['Bank Master Checking']

sheet4.write('A1', 'Bank Master Data Quality Checking', header_format)
sheet4.write("A2", "1) TOTAL RECORDS BANK MASTER (SALESFORCES) : " , body_format)
sheet4.write("B2", len(df_bank_sf_new.index), body_format)
sheet4.write("A3", "  UNIQUE VALUES : " , body_format)
pv_bank_sf_unique = pd.DataFrame(df_bank_sf_new.nunique())
pv_bank_sf_unique.to_excel(writer, sheet_name = "Bank Master Checking", startrow= 5)  

sheet4.write('A14', '  TOTAL SF BANK NAME MASTER DUPLICATE :' , body_format)
sheet4.write('B14' , df_bank_sf_new["SF bank name"].duplicated().sum() , body_format)
#df_dup_bank_sf_new = df_bank_sf_new[df_bank_sf_new.duplicated()]

sheet4.write('A16','2) TOTAL RECORDS BANK MASTER (BILLING TOOL) : ' , body_format)
sheet4.write('B16', len(df_bank_billing_new.index), body_format)
sheet4.write("A17", "  UNIQUE VALUES : " , body_format)
pv_unique_bank_billing_new = pd.DataFrame(df_bank_billing_new.nunique())
pv_unique_bank_billing_new.to_excel(writer, sheet_name = "Bank Master Checking", startrow= 19)

sheet4.write("A30" , "TOTAL DUPLICATE BT Bank name : " , body_format)
sheet4.write("B30", df_bank_billing_new["BT bank name"].duplicated().sum(), body_format)
##df_bank_billing_new[df_bank_sf_new.duplicated()]

sheet4.write("A31", "3) AFTER REMOVE DUPLICATE ROWS, IF THE BANK NAMES DO NOT UNIQUE , PLEASE CHECK BANK MASTER FILE !!!. " , body_format)
sheet4.write("A32", "TOTAL DUPLICATE BT BANK NAME : " , body_format)
sheet4.write("B32", df_bank_billing_unique["BT bank name"].duplicated().sum(), body_format)
sheet4.write("A34", "TOTAL DUPLICATE SF BANK NAME : " , body_format)
sheet4.write("B34", df_bank_sf_unique["SF bank name"].duplicated().sum() , body_format)
### DROP DUPLICATE ROWS ####
#df_bank_sf_unique = df_bank_sf_new.drop_duplicates()
#df_bank_billing_unique = df_bank_billing_new.drop_duplicates()
### CHECK WHETHER STILL HAVE DUPLICATE BANK NAME OR NOT ####


0

In [106]:
print("1) TOTAL RECORDS AFTER MATCHING : " + str(len(df_bill_sf_merge.index)))
print("2) TOTAL MATCH SALESFORCE ID : " + str(len(df_bill_sf_merge[(df_bill_sf_merge["GRID"].isnull())].index)))
print("3) NOT FOUND IN BILLING TOOL : " + str(len(df_bill_sf_merge["salesforce_id"].isnull().index)))
print("4) NOT FOUND IN SALESFORCE : " + str(len(df_bill_sf_merge[(df_bill_sf_merge["salesforce_id"].isnull())].index)))
print("5) NOT FOUNT IN SALESFORCE AND  BILLING TOO: " +str(len(df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].isnull() & df_bill_sf_merge["GRID"].isnull())].index))) # THERE ARE NO salesforce_id in billing tool
print("6) MATCHING TAX ID")
no_match_bf_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["Tax Number"] == df_bill_sf_merge["tax_id"])].count()
print("TOTAL MATCH TAX ID BEFORE CLEANNING : " + str(no_match_bf_cleaning))
no_match_af_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["new_tax_id"] == df_bill_sf_merge["new_tax_id_B"])].count()
print("TOTAL MATCH TAX ID AFTER CLEANNING : " + str(no_match_af_cleaning))

print("7) MATCHING ACC NO CLEANING " )
no_acc_match_bf_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["Bank Account Number"] == df_bill_sf_merge["bank_account_number"])].count()
print("TOTAL MATCH ACC NO BEFORE CLEANNING : " + str(no_acc_match_bf_cleaning)) 
no_acc_match_af_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["new_acc_no"] == df_bill_sf_merge["new_acc_no_B"])].count()
print("TOTAL MATCH ACC NO AFTER CLEANNING : " + str(no_acc_match_af_cleaning))
print("8) STATUS ACCOUNT NO : ")
print(df_bill_sf_merge.groupby("status_acc_no").count()[['vendor_code']])
print("9) STATUS TAX ID : ")
print(df_bill_sf_merge.groupby("status_tax_id").count()[['vendor_code']])

1) TOTAL RECORDS AFTER MATCHING : 152722
2) TOTAL MATCH SALESFORCE ID : 29247
3) NOT FOUND IN BILLING TOOL : 152722
4) NOT FOUND IN SALESFORCE : 2520
5) NOT FOUNT IN SALESFORCE AND  BILLING TOO: 180
6) MATCHING TAX ID
TOTAL MATCH TAX ID BEFORE CLEANNING : 5648
TOTAL MATCH TAX ID AFTER CLEANNING : 72399
7) MATCHING ACC NO CLEANING 
TOTAL MATCH ACC NO BEFORE CLEANNING : 75
TOTAL MATCH ACC NO AFTER CLEANNING : 69934
8) STATUS ACCOUNT NO : 
                                         vendor_code
status_acc_no                                       
0                                                 37
CHECK                                          69822
CHECK - SWAP BANK ACC NO WITH BANK NAME        13045
CHECK - ZERO LEADING                           20517
MISSING BL                                         4
MISSING BOTH                                       0
MISSING SF                                     38443
NOT MATCH                                       8439
OK                           

**Print Merging Result to Excel**

In [107]:

df3 = pd.DataFrame(data = None)
df3.to_excel(writer, sheet_name = 'Merging Result' , startrow = 1 )
sheet3 = writer.sheets['Merging Result']

sheet3.write("A1" , "MATCHING SUMMARY", header_format)
sheet3.write("A2","1) TOTAL RECORDS AFTER MATCHING : " , body_format)
sheet3.write("B2" ,(len(df_bill_sf_merge.index)), body_format)
sheet3.write("A3","2) TOTAL MATCH SALESFORCE ID : ", body_format)
sheet3.write("B3",(df_bill_sf_merge.salesforce_id[(df_billing_new["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull())].count()),body_format)
sheet3.write("A4", "3) NOT FOUND IN BILLING TOOL : ", body_format)
sheet3.write("B4" ,(df_bill_sf_merge.salesforce_id[(df_billing_new["salesforce_id"].isnull() & df_bill_sf_merge["GRID"].notnull())].count()), body_format)
sheet3.write("A5", "4) NOT FOUND IN SALESFORCE : " , body_format)
sheet3.write("B5" , (df_bill_sf_merge.salesforce_id[(df_billing_new["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].isnull())].count()), body_format)
sheet3.write("A6", "5) NOT FOUND IN SALESFORCE AND BILLING TOOL:  ** VENDOR_CODE in billing tool exist, but salesforce_id is null ", body_format)
sheet3.write("B6", (len(df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].isnull() & df_bill_sf_merge["GRID"].isnull())].index)), body_format)
sheet3.write("A7", "6) MATCHING TAX ID", body_format)
no_match_bf_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["Tax Number"] == df_bill_sf_merge["tax_id"])].count()

sheet3.write("A8", "TOTAL MATCH TAX ID BEFORE CLEANNING : " , body_format)
sheet3.write("B8" ,(no_match_bf_cleaning), body_format)

no_match_af_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["new_tax_id"] == df_bill_sf_merge["new_tax_id_B"])].count()
sheet3.write("A9", "TOTAL MATCH TAX ID AFTER CLEANNING : " , body_format )
sheet3.write("B9", (no_match_af_cleaning), body_format)

sheet3.write("A10","7) MATCHING ACC NO CLEANING " , body_format)
no_acc_match_bf_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["Bank Account Number"] == df_bill_sf_merge["bank_account_number"])].count()
sheet3.write("A11","TOTAL MATCH ACC NO BEFORE CLEANNING : ", body_format)
sheet3.write("B11" ,(no_acc_match_bf_cleaning), body_format)
no_acc_match_af_cleaning = df_bill_sf_merge.salesforce_id[(df_bill_sf_merge["salesforce_id"].notnull() & df_bill_sf_merge["GRID"].notnull()) & (df_bill_sf_merge["new_acc_no"] == df_bill_sf_merge["new_acc_no_B"])].count()
sheet3.write("A12","TOTAL MATCH ACC NO AFTER CLEANNING : ", body_format)
sheet3.write("B12" ,(no_acc_match_af_cleaning), body_format)
sheet3.write("A18", "TOTAL RECORDS BY ACCOUNT_NO STAUS :" , body_format)
pv_status_acc = (df_bill_sf_merge.groupby("status_acc_no").count()[['vendor_code']])
pv_status_acc.to_excel(writer, sheet_name = 'Merging Result', startrow = 21)

sheet3.write("A34", "TOTAL RECORDS BY TAX ID STATUS : " , body_format)
pv_tax_status = (df_bill_sf_merge.groupby("status_tax_id").count()[['vendor_code']])
pv_tax_status.to_excel(writer, sheet_name = 'Merging Result' , startrow = 35)

sheet3.write("A44", "TOTAL RECORDS BY ACCOUNT NAME STATUS" , body_format)
pv_acc_name_match = pd.DataFrame(df_bill_sf_merge.groupby("status_account_master_matching").count()[['GRID']])
pv_acc_name_match.to_excel(writer, sheet_name = 'Merging Result' , startrow = 45)


sheet3.write("A54", "TOTAL RECORDS BY COMMISSION STATUS" , body_format)
pv_comm_match = pd.DataFrame(df_bill_sf_merge.groupby("status_commission").count()[['GRID']])
pv_comm_match.to_excel(writer, sheet_name = 'Merging Result' , startrow = 55)



sheet3.write("A60" , "**STATUS DEFINITION" , body_format)
sheet3.write("A61" , "1) status_tax_id" , body_format)
sheet3.write("A62", "    OK : Pass without cleaning. Original data has the correct format and can be matched without cleaning.")
sheet3.write("A63", "    CHECK : Formats are corrected after cleaning")
sheet3.write("A64", "    CHECK - ZERO LEADING : Cleaning data will be match after adding the zero leading. SF or BL need to add zero (1 or more digits). For this scenario, the cleaned data which have longer length should be used.")
sheet3.write("A65", "    NOT MATCH : data cannot be matched.")
sheet3.write("A66", "    0 : data cannot be matched.")
sheet3.write("A67", "    MISSING SF : data cannot be matched , but the information is still exist in Billing Tool")
sheet3.write("A68", "    MISSING BL : data cannot be matched , but the information is still exist in Salesforce")
sheet3.write("A69", "    MISSING BOTH : data cannot be matched due to no information available for both Salesforce and Billing Tool")


sheet3.write("A71" , "2) status_acc_no" , body_format)
sheet3.write("A72", "    OK : Pass without cleaning. Original data has the correct format and can be matched without cleaning.")
sheet3.write("A73", "    CHECK : Formats are corrected after cleaning")
sheet3.write("A74", "    CHECK - ZERO LEADING : Cleaning data will be match after adding the zero leading. SF or BL need to add zero (1 or more digits). For this scenario, the cleaned data which have longer length should be used.")
sheet3.write("A75", "    NOT MATCH : data cannot be matched.")
sheet3.write("A76", "    0 : data cannot be matched.")
sheet3.write("A77", "    MISSING SF : data cannot be matched , but the information is still exist in Billing Tool")
sheet3.write("A78", "    MISSING BL : data cannot be matched , but the information is still exist in Salesforce")
sheet3.write("A79", "    MISSING BOTH : data cannot be matched due to no information available for both Salesforce and Billing Tool")
sheet3.write("A80", "    CHECK - SWAP BANK ACC NO WITH BANK NAME : Bank name and bank account no. were switched in Billing Tool. After swap the two columns and clean the data, the data can be marched. ")



sheet3.write("A81" , "3) status_account_master_matching" , body_format)
sheet3.write("A72", "    OK : Bank name in Billing Tool and Salesforce can be match after joining the bank info")
sheet3.write("A73", "    CHECK : Formats are corrected after cleaning")
sheet3.write("A74", "    NOT MATCH : After matching with the bank name file, the data cannot be matched.")
sheet3.write("A75", "    MISSING SF BANK MASTER : data cannot be matched , but the information is still exist in Billing Tool")
sheet3.write("A76", "    MISSING BL BANK MASTER: data cannot be matched , but the information is still exist in Salesforce")
sheet3.write("A77", "    MISSING BOTH : data cannot be matched due to no information available for both Salesforce Bankname cleaning and Billing Tool bankname cleaning file ")
sheet3.write("A78", "    MISSING BOTH - BUT BOTH SF BL HAVE BANK NAME : The two bank name master file does not have the information, However, from the SF and Billing tools still have the bank name information. This might need to re-check the bank master file. ")

sheet3.write("A81" , "4) status_commission" , body_format)
sheet3.write("A82", "    MATCH - BOTH SIDE HAVE THE SAME TIERS VALUES (> 1 TIERS) : All commission tier values for the vendors which have the commission tier are match and the number of teirs are not exceed 5 tiers.")
sheet3.write("A83", "    MATCH - STANDARD COMMISSION : Commission value in standard commission vendors are match both sides.")
sheet3.write("A84", "    MISMATCH - NO.TIERS (BOTH) > 5 : Number of commission tiers from both sides are exceed 5 tiers. Require to re-check for both sides.")
sheet3.write("A85", "    MISMATCH - NO.TIERS (SF) > 5 : Number of commission tiers from Salesdforce are exceed 5 tiers. Require to re-check.")
sheet3.write("A86", "    MISMATCH - NO.TIERS (BL) > 5 : Number of commission tiers from Billing Tool are exceed 5 tiers. Require to re-check.")
sheet3.write("A87", "    MISSING - BOTH SF AND BL : No commission info from both side. Need to re-check why the vendor does not have any commission values.")
sheet3.write("A88", "    MISSING FROM SF : There is commission value in Billing Tool, but no data in Salesforces.")
sheet3.write("A89", "    MISSING FROM BL : There is commission value in Salesforces, but no data in Billing Tool.")



0

In [108]:
df_bill_sf_merge.info()

df_raw = df_bill_sf_merge[[
                           'GRID',
                           'Bank Name',
                           'Bank Account Number',
                           'Commission_tier',
                           'total_tiers',
                           'Tax Number',
                           'len_tax_id',
                           'is_tax_contain_char',
                           'is_tax_contain_space',
                           'is_tax_id_len_correct',
                           'len_acc_no',
                           'is_acc_no_len_correct',
                           'is_acc_contain_char',
                           'is_acc_contain_space',
                           'new_tax_id',
                           'new_acc_no',
                           'new_len_tax_id',
                           'new_len_acc_no',
                           'is_new_tax_id_len_correct',
                           'is_new_acc_no_len_correct',
                           'is_tax_id_null',
                           'is_acc_no_null',
                           'is_tax_id_pass_at_the_beginning',
                           'is_acc_no_pass_at_the_beginning',
                           'is_no_tier_correct',
                           'is_tax_id_pass_after_cleaning',
                           'tax_id_status',
                           'client_name',
                           'tax_id',
                           'bank_account_number',
                           'bank_name',
                           'vendor_code',
                           'vendor_name',
                           'salesforce_id',
                           'sf_account_id',
                           'stv',
                           'total_tiers_b',
                           'bank_account_number_swap',
                           'bank_name_swap',
                           'len_tax_id_B',
                           'is_tax_contain_char_B',
                           'is_tax_contain_space_B',
                           'is_tax_id_len_correct_B',
                           'len_acc_no_B',
                           'is_acc_no_len_correct_B',
                           'is_acc_contain_char_B',
                           'is_acc_contain_space_B',
                           'new_tax_id_B',
                           'new_acc_no_B',
                           'new_len_tax_id_B',
                           'new_len_acc_no_B',
                           'is_new_tax_id_len_correct_B',
                           'is_new_acc_no_len_correct_B',
                           'is_tax_id_null_B',
                           'is_acc_no_null_B',
                           'is_salesforce_id_null',
                           'is_sf_account_id_null',
                           'bank_account_number_swap_clean',
                           'len_acc_no_swap',
                           'is_len_acc_no_swap_correct',
                           'is_teir_excceed_5',
                           'is_tax_id_pass_after_cleaning_B',
                           'tax_id_status_B',
                           'acc_no_status_B',
                           'sum_account_checking_B',
                           'sum_tax_id_checking_B',
                           'SF bank name',
                           'clean_name_sf',
                           'chk_sf',
                           'name_for_upload_sf',
                           'BT bank name',
                           'clean_name_B',
                           'chk_B',
                           'name_for_upload_B',
                           'BT_bank_name_BL_BANK_SWAP',
                           'clean_name_B_BL_BANK_SWAP',
                           'chk_B_BL_BANK_SWAP',
                           'name_for_upload_B_BL_BANK_SWAP',
                           'is_bank_acc_swap_match',
                           'is_bank_name_swap_match',
                           'new_tax_id_13_digit',
                           'new_tax_id_B_13_digit',
                           'new_acc_no_13_digit',
                           'new_acc_no_B_13_digit',
                           'is_all_tier_match',
                           'status_acc_no',
                           'status_tax_id',
                           'status_account_master_matching',
                           'status_commission'
                           ]]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152722 entries, 0 to 152721
Data columns (total 99 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   GRID                              123475 non-null  object 
 1   Bank Name                         113032 non-null  object 
 2   Bank Account Number               113015 non-null  object 
 3   Commission_tier                   123184 non-null  object 
 4   total_tiers                       123184 non-null  float64
 5   Tax Number                        123475 non-null  object 
 6   len_tax_id                        123475 non-null  float64
 7   is_tax_contain_char               123475 non-null  float64
 8   is_tax_contain_space              123475 non-null  float64
 9   is_tax_id_len_correct             123475 non-null  float64
 10  len_acc_no                        123475 non-null  float64
 11  is_acc_no_len_correct             123475 non-null  f

In [109]:
#EXPORT RAW DATA TO CSV
df_bill_sf_merge.to_csv(r'SF_BILLING_RAW.csv', index = False)


In [110]:
df_r = pd.DataFrame(data = None)
df_r.to_excel(writer, sheet_name = 'RAW DATA' , startrow = 1 )
sheet5 = writer.sheets['RAW DATA']
df_raw.to_excel(writer, sheet_name = 'RAW DATA', startrow = 0)
writer.save()

**STATUS AND MEANING**

**OK**  = BT IS THE SAME AS SF 

**CHECK** = Basically the information needs edit but 
For bank account no. BT = SF, BUT field does not have 10-15 digits
For tax ID BT = SF, BUT field does not have 10-13 digits

**NOT MATCH** = BT <> SF (need to check for correct value in documents / contracts); but also

**MISSING SF** = SF field missing, BT field available

**MISSING BT** =BT field missing, SF field available

**MISSING BOTH** = Both fields are blank
